In [1]:
#pip install biopython

In [2]:
import pandas as pd
import json
import re
import numpy as np
import pickle

In [3]:
# Biopython entrez

In [4]:
from Bio import Entrez

def search(query): # scraping
    Entrez.email = 'xji1@dons.usfca.edu'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance',
                            retmax='310',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):  
    ids = ','.join(id_list)
    Entrez.email = 'xji1@dons.usfca.edu'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

if __name__ == '__main__': # parsing 
    results = search(['randomized control* [abstract] AND (insomnia OR sleep) AND (depressi* OR anxiety) AND (Herb OR mind-body OR acup* OR music OR mindfulness OR meditation OR Tai ji OR tai chi OR qigong OR yoga OR hypnotherapy OR massage OR manipulation OR traditional Chinese Medicine OR homeopathy OR relaxation OR guided imagery) AND 2010:2017[Date - Publication]'])
    id_list = results['IdList'] # a list of pubUIDs from fetching
    papers = fetch_details(id_list)
    #for i, paper in enumerate(papers['PubmedArticle']):
        #print("%d) %s" % (i+1, paper['MedlineCitation']['Article']))
    print(id_list)

['26880252', '28576734', '27543065', '28489508', '29222705', '25142571', '23337557', '27502810', '28380067', '25124456', '28427459', '26115880', '26362690', '27527892', '20860572', '28884877', '26424362', '25686304', '21934474', '24440643', '25843539', '27852282', '28431122', '22903446', '28899535', '22350253', '28899206', '25342100', '26361642', '27511921', '27451854', '29229613', '28806907', '26734485', '21629370', '29101843', '27775416', '24491199', '21676255', '26433466', '27467589', '26498238', '26132682', '26745049', '28286776', '25132206', '23286297', '27895522', '22623888', '24045673', '22621705', '26111942', '29458940', '28735173', '28153215', '25986296', '28110112', '22017318', '22832540', '28183071', '22943846', '22048261', '24577747', '23629959', '23782864', '26228466', '21246844', '22808932', '28686520', '29164609', '26844196', '25543421', '25873295', '25537522', '23982113', '22909345', '25271149', '22840618', '26537175', '22417316', '23338490', '29389474', '26802824', '26

In [5]:
len(id_list)

309

from Bio import Entrez
Entrez.email = 'xji1@dons.usfca.edu'
handle = Entrez.efetch(db="pubmed", id="22459648",
                       rettype = 'medline', retmode="text")
print(handle.read())

In [6]:
CAMJournal=[]
for i, paper in enumerate(papers['PubmedArticle']):
    CAMJournal.append(paper['MedlineCitation']['Article']['Journal']['ISOAbbreviation'])
CAMJournal
len(CAMJournal)

308

In [7]:
#CAMAb=[]
#for i, paper in enumerate(papers['PubmedArticle']):
    #try:
        #CAMAb.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
    #except KeyError:
        #pass
#len(CAMAb)

In [8]:
CAMAb=[]
for i, paper in enumerate(papers['PubmedArticle']):
    try:
        if 'AbstractText' in paper['MedlineCitation']['Article']['Abstract']:
            CAMAb.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
        else:
            CAMAb.append('NaN')
            
    except KeyError as e:
        CAMAb.append('NaN')
        
CAMAb

len(CAMAb)

308

In [9]:
CAMtitles=[]
for i, paper in enumerate(papers['PubmedArticle']):
    CAMtitles.append(paper['MedlineCitation']['Article']['ArticleTitle'])
CAMtitles
len(CAMtitles)

308

In [10]:
combdata= list(zip(CAMtitles, CAMJournal)) 
df0 = pd.DataFrame(combdata,columns = ['Title', 'JournalInfo']) 
len(df0)

308

In [11]:
combdata= list(zip(CAMtitles, CAMJournal, CAMAb)) 
df = pd.DataFrame(combdata,columns = ['Title', 'JournalInfo','Abstract']) 
len(df)

308

In [12]:
df['Abstract'][1]

[StringElement('A growing number of cancer survivors suffer high levels of distress, depression and stress, as well as sleep disturbance, pain and fatigue. Two different mind-body interventions helpful for treating these problems are Mindfulness-Based Cancer Recovery (MBCR) and Tai Chi/Qigong (TCQ). However, while both interventions show efficacy compared to usual care, they have never been evaluated in the same study or directly compared. This study will be the first to incorporate innovative design features including patient choice while evaluating two interventions to treat distressed cancer survivors. It will also allow for secondary analyses of which program best targets specific symptoms in particular groups of survivors, based on preferences and baseline characteristics.', attributes={'Label': 'PURPOSE'}),
 StringElement('The design is a preference-based multi-site randomized comparative effectiveness trial. Participants (N=600) with a preference for either MBCR or TCQ will rece

In [13]:
df['Abstract'] = df.Abstract.apply(lambda x: ' '.join(x))

In [14]:
def remove_brackets(x):
    remove = '<>()[]{}"'
    p = re.compile("[" + re.escape(remove) + "]")
    return p.sub("",x)

In [15]:
df['JournalInfo'] = df.JournalInfo.apply(lambda x: json.dumps(x))
df['Title'] = df.Title.apply(lambda x: json.dumps(x))
type(df.JournalInfo[0])
type(df.Title[0])

str

In [16]:
df['JournalInfo'] = df.JournalInfo.apply(lambda x: remove_brackets(x))
type(df.Abstract[0])

str

In [17]:
#df['Title'] = df.Title.apply(lambda x: json.dumps(x))
#type(df.Title[0])

In [18]:
df.head(20)

,Title,JournalInfo,Abstract
0,"""Effects of Acupuncture, Tuina, Tai Chi, Qigon...",J Pain Symptom Manage,Most cancer patients suffer from both the dise...
1,"""Protocol for the MATCH study (Mindfulness and...",Contemp Clin Trials,A growing number of cancer survivors suffer hi...
2,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...
3,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...
4,"""Tai Chi and Qigong for cancer-related symptom...",J Cancer Surviv,This study aims to summarize and critically ev...
5,"""Cognitive behavioral therapy vs. Tai Chi for ...",Sleep,To investigate the comparative efficacy of cog...
6,"""Tai chi/yoga reduces prenatal depression, anx...",Complement Ther Clin Pract,Ninety-two prenatally depressed pregnant women...
7,"""Efficacy of rehabilitation with Tai Ji Quan i...",Complement Ther Clin Pract,Fibromyalgia Syndrome (FMS) is characterized b...
8,"""Does Tai Chi relieve fatigue? A systematic re...",PLoS One,Fatigue is not only a familiar symptom in our ...
9,"""Randomized controlled trial of Qigong/Tai Chi...",Ann Behav Med,Many breast cancer survivors experience fatigu...


In [19]:
#df.head()

In [20]:
# Write a new columne 'Til' : labelling based on title (lable papers titled with 'protocol','systematic review','meta-analysis' or 'guideline' as 0, else 1)

In [21]:
df['TiL'] = np.where(df['Title'].str.lower().str.contains("meta-analysis|systematic review|protocol|guideline|review"), 0, 1)
df.head()

,Title,JournalInfo,Abstract,TiL
0,"""Effects of Acupuncture, Tuina, Tai Chi, Qigon...",J Pain Symptom Manage,Most cancer patients suffer from both the dise...,0
1,"""Protocol for the MATCH study (Mindfulness and...",Contemp Clin Trials,A growing number of cancer survivors suffer hi...,0
2,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...,1
3,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1
4,"""Tai Chi and Qigong for cancer-related symptom...",J Cancer Surviv,This study aims to summarize and critically ev...,0


In [22]:
type(df['TiL'][0])

numpy.int32

In [23]:
df.to_pickle('C:/Users/CAM1-308.pkl')
#df.to_pickle('C:/Users/kirak/Documents/School/USF/HS651/CAM1.pkl')

In [24]:
#df = pd.read_pickle('C:/Users/CAM1-1.pkl')
#df = pd.read_pickle('C:/Users/kirak/Documents/School/USF/HS651/CAM1.pkl')

In [25]:
duplicateRow = df[df.duplicated(['Title'])]
print(duplicateRow)

                                                 Title  \
278  "Physical activity and exercise for chronic pa...   
287   "Exercise therapy for chronic fatigue syndrome."   
288   "Exercise therapy for chronic fatigue syndrome."   
292   "Exercise therapy for chronic fatigue syndrome."   
294   "Exercise therapy for chronic fatigue syndrome."   

                    JournalInfo  \
278  Cochrane Database Syst Rev   
287  Cochrane Database Syst Rev   
288  Cochrane Database Syst Rev   
292  Cochrane Database Syst Rev   
294  Cochrane Database Syst Rev   

                                              Abstract  TiL  
278  Chronic pain is defined as pain lasting beyond...    0  
287  Chronic fatigue syndrome (CFS) is characterise...    1  
288  Chronic fatigue syndrome (CFS) is characterise...    1  
292  Chronic fatigue syndrome (CFS) is characterise...    1  
294  Chronic fatigue syndrome (CFS) is characterise...    1  


In [26]:
df['Abstract'][278]

"Chronic pain is defined as pain lasting beyond normal tissue healing time, generally taken to be 12 weeks. It contributes to disability, anxiety, depression, sleep disturbances, poor quality of life, and healthcare costs. Chronic pain has a weighted mean prevalence in adults of 20%.For many years, the treatment choice for chronic pain included recommendations for rest and inactivity. However, exercise may have specific benefits in reducing the severity of chronic pain, as well as more general benefits associated with improved overall physical and mental health, and physical functioning.Physical activity and exercise programmes are increasingly being promoted and offered in various healthcare systems, and for a variety of chronic pain conditions. It is therefore important at this stage to establish the efficacy and safety of these programmes, and furthermore to address the critical factors that determine their success or failure. To provide an overview of Cochrane Reviews of adults wit

In [27]:
df['Title'][278]

'"Physical activity and exercise for chronic pain in adults: an overview of Cochrane Reviews."'

In [28]:
df1_1 = pd.read_pickle('C:/Users/CAM1-1.pkl')

In [61]:
df1_1.head()

,Title,JournalInfo,Abstract,TiL
0,"""Effects of Acupuncture, Tuina, Tai Chi, Qigon...",J Pain Symptom Manage,Most cancer patients suffer from both the dise...,0
1,"""Protocol for the MATCH study (Mindfulness and...",Contemp Clin Trials,A growing number of cancer survivors suffer hi...,0
2,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...,1
3,"""Tai Chi and Qigong for cancer-related symptom...",J Cancer Surviv,This study aims to summarize and critically ev...,0
4,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1


In [30]:
len(df1_1)

306

In [38]:
len(df)

308

In [42]:
df['Abstract'].str.split(" ").str.len().sort_values()

97       3
162      3
6       79
99      99
55     101
      ... 
288    976
287    976
292    976
285    978
294    978
Name: Abstract, Length: 308, dtype: int64

In [54]:
df['Abstract'][97]

'N a N'

In [44]:
df['Abstract'][162]

'N a N'

In [58]:
df_Abs = df[df["TiL"] == 1]

In [59]:
df_Abs.head()

,Title,JournalInfo,Abstract,TiL
2,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...,1
3,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1
5,"""Cognitive behavioral therapy vs. Tai Chi for ...",Sleep,To investigate the comparative efficacy of cog...,1
6,"""Tai chi/yoga reduces prenatal depression, anx...",Complement Ther Clin Pract,Ninety-two prenatally depressed pregnant women...,1
7,"""Efficacy of rehabilitation with Tai Ji Quan i...",Complement Ther Clin Pract,Fibromyalgia Syndrome (FMS) is characterized b...,1


In [45]:
#manully input value to new colunm 'AbL' (labelling based on abstract)

In [46]:
df_Abs.loc[:,'AbL'] = 1

C:\anocoda\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\anocoda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [47]:
df_Abs = df_Abs.reset_index(drop =  True)

In [48]:
df_Abs.head()

,Title,JournalInfo,Abstract,TiL,AbL
0,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...,1,1
1,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1,1
2,"""Cognitive behavioral therapy vs. Tai Chi for ...",Sleep,To investigate the comparative efficacy of cog...,1,1
3,"""Tai chi/yoga reduces prenatal depression, anx...",Complement Ther Clin Pract,Ninety-two prenatally depressed pregnant women...,1,1
4,"""Efficacy of rehabilitation with Tai Ji Quan i...",Complement Ther Clin Pract,Fibromyalgia Syndrome (FMS) is characterized b...,1,1


In [49]:
len(df_Abs)

244

In [51]:
#df_Abs['Abstract'][0]: protocol

'Breast cancer survivors (BCSs) often report fatigue that persists for years following treatment. Despite a growing body of evidence for meditative movement practices to improve symptoms among BCSs, few studies have explored using Qigong/Tai Chi to reduce fatigue. Additionally, few have examined the biological mechanisms through which fatigue may be reduced using Qigong/Tai Chi. We will recruit 250 fatigued, post-menopausal women diagnosed with breast cancer (stage 0-III), between 6months and 5years past primary treatment and randomize to a standardized Qigong/Tai Chi Easy (QG/TCE) intervention, a "sham" Qigong group (movements without a focus on the breath and meditative state) (SQG), or an educational support (ES) group. The primary outcome (fatigue), secondary outcomes (anxiety, depression, sleep quality, cognitive function, physical activity), and a biomarker of HPA axis dysregulation (diurnal cortisol) will be assessed at baseline, post-intervention and 6months postintervention, a

In [52]:
df_Abs.at[0,'AbL'] = 0

In [60]:
#df_Abs['Abstract'][7]: design and method

'"Design and methods of the Gentle Cardiac Rehabilitation Study--A behavioral study of tai chi exercise for patients not attending cardiac rehabilitation."'

In [61]:
df_Abs.at[7,'AbL'] = 0

In [62]:
#df_Abs['Abstract'][8]:plan

'Persons with chronic obstructive pulmonary disease (COPD) have reduced exercise capacity and levels of physical activity. Supervised, facility-based pulmonary rehabilitation programs improve exercise capacity and reduce dyspnea, but novel long-term strategies are needed to maintain the benefits gained. Mind-body modalities such as Tai Chi which combine aerobic activity, coordination of breathing, and cognitive techniques that alleviate the physical inactivity, dyspnea, and anxiety and depression that are the hallmarks of COPD are promising strategies. We have designed a randomized controlled study to examine whether Tai Chi will maintain exercise capacity in persons with COPD who have recently completed a supervised pulmonary rehabilitation program, compared to standard care. The primary outcome is 6-min walk test distance at 6 months. Secondary outcomes include health-related quality of life, dyspnea, mood, occurrence of acute exacerbations, engagement in physical activity, exercise 

In [63]:
df_Abs.at[8,'AbL'] = 0

In [64]:
#df_Abs['Abstract'][9] : qualitative reivew of RCT

"Physicians encounter various symptom-based complaints each day. While physicians strive to support patients with chronic illnesses, evidence indicates that patients who are actively involved in their health care have better health outcomes and sometimes lowers costs. This article is to analyze how patient engagement is described when complex interventions such as Tai Chi were delivered in Randomized Controlled clinical Trials (RCTs). It reviews the dynamic patient- physician relationship in chronic illness management and to illustrate the patient engagement process, using Tai Chi as an example intervention. RCTs are considered the gold standard in clinical research. This study is a qualitative analysis of RCTs using Tai Chi as an intervention. A systematic literature search was performed to identify quality randomized controlled clinical trials that investigated the effects of Tai Chi. Selected clinical trials were classified according to research design, intervention style, patient e

In [65]:
df_Abs.at[9,'AbL'] = 0

In [75]:
#df_Abs['Abstract'][17]: adolescent participants

'The aim of this study was to test whether a cognitive behavioral and mindfulness-based group sleep intervention would improve sleep and anxiety on school nights in a sample of at-risk adolescents. We also examined whether benefits to sleep and anxiety would be mediated by improvements in sleep hygiene awareness and presleep hyperarousal. Secondary analysis of a randomized controlled trial conducted with 123 adolescent participants (female = 60%; mean age = 14.48) who had high levels of sleep problems and anxiety symptoms. Participants were randomized into a sleep improvement intervention (n = 63) or active control "study skills" intervention (n = 60). Preintervention and postintervention, participants completed the Pittsburgh Sleep Quality Index (PSQI), Spence Children\'s Anxiety Scale (SCAS), Sleep Beliefs Scale (SBS), and Presleep Hyperarousal Scale (PSAS) and wore an actiwatch and completed a sleep diary for five school nights. The sleep intervention condition was associated with s

In [76]:
df_Abs.at[17,'AbL'] = 0

In [79]:
#df_Abs['Abstract'][20]:secondary analysis

'This secondary analysis examined changes in sleep quality associated with participation in a Mindfulness-Based Stress Reduction (MBSR) program among healthy older adults. Data were collected at baseline, 8-weeks post-treatment, and a 6-month follow-up from adults aged ≥ 65 ( N = 200), randomly assigned to MBSR or a waitlist control. Group differences were examined using mixed analysis of covariance with repeated measures on the total Pittsburgh Sleep Quality Index (PSQI) score. A small-sized, significant effect was found on overall sleep among MBSR participants with baseline PSQI scores > 5, indicative of a sleep disturbance, F(2, 80) = 4.32, p = .02, η<sub>p</sub><sup>2</sup> = .05. A medium-sized, significant effect was found for MBSR participants with baseline PSQI scores ≥ 10, F(2, 28) = 3.13, p = .04, η<sub>p</sub><sup>2</sup> = .10. These findings indicate that improved sleep quality for older adults who have higher levels of sleep disturbance may be associated with participatio

In [80]:
df_Abs.at[20,'AbL'] = 0

In [86]:
#df_Abs['Abstract'][26]: adolescents participants

'The aim of this study was to test whether a cognitive-behavioral and mindfulness-based group sleep intervention would improve behavior problems in at-risk adolescents, and whether these improvements were specifically related to improvements in sleep. Secondary analysis of a randomized controlled trial conducted with 123 adolescent participants (female\xa0=\xa060%; mean age\xa0=\xa014.48, range 12.04-16.31 years) who had high levels of sleep problems and anxiety symptoms. Participants were randomized into either a sleep improvement intervention (n\xa0=\xa063) or an active control "study skills" intervention (n\xa0=\xa060). Participants completed sleep and behavior problems questionnaires, wore an actiwatch and completed a sleep diary for five school nights, both before and after the intervention. Parallel multiple mediation models showed that postintervention improvements in social problems, attention problems, and aggressive behaviors were specifically mediated by moderate improvement

In [87]:
df_Abs.at[26,'AbL'] = 0

In [88]:
#df_Abs['Abstract'][27]: adolescent

"Sleep problems are a major risk factor for the emergence of mental health problems in adolescence. The aim of this study was to investigate the post intervention effects of a cognitive-behavioral/mindfulness-based group sleep intervention on sleep and mental health among at-risk adolescents. A randomized controlled trial (RCT) was conducted across High schools in Melbourne, Australia. One hundred forty-four adolescents (aged 12-17 years) with high levels of anxiety and sleeping difficulties, but without past or current depressive disorder, were randomized into either a sleep improvement intervention or an active control 'study skills' intervention. Both programs consisted of 7 90-min-long group sessions delivered over 7 weeks. One hundred twenty-three participants began the interventions (female = 60%; mean age = 14.48, SD = 0.95), with 60 in the sleep condition and 63 in the control condition. All participants were required to complete a battery of mood and sleep questionnaires, 7 da

In [89]:
df_Abs.at[27,'AbL'] = 0

In [92]:
#df_Abs['Abstract'][29]: CBT vs. relaxation active control

'To investigate the effects of group CBT for insomnia (CBT-I) on insomnia and depressive symptomatology in a comorbid sample through a randomised controlled trial with a 6 month follow-up. 64 participants were recruited through advertisements and randomised to receive CBT-I or an active control (relaxation training: RT) during four group sessions. Insomnia Severity Index and BDI-II were the primary outcome measures, assessed pre-treatment, post-treatment and at 6 month follow-up. Insomnia and depressive diagnoses, and functional impairment were assessed before and after treatment, whereas sleep diary data was gathered continuously from one week before treatment until after treatment. CBT-I was more efficient than RT in reducing insomnia severity and equally effective in reducing depressive symptoms, although CBT-I was associated with a higher proportion of remitted persons than RT, regarding both insomnia and depression diagnoses. Also, CBT-I was associated with less functional impairm

In [93]:
df_Abs.at[29,'AbL'] = 0

In [94]:
#df_Abs['Abstract'][30]:waking hypnosis with cognitive-behavioral therapy (VMWH-CBT)

"This study evaluated the efficacy of an intervention combining the Valencia model of waking hypnosis with cognitive-behavioral therapy (VMWH-CBT) in managing cancer-related pain, fatigue, and sleep problems in individuals with active cancer or who were post-treatment survivors. We hypothesized that four sessions of VMWH-CBT would result in greater improvement in participants' symptoms than four sessions of an education control intervention. Additionally, we examined the effects on several secondary outcome domains that are associated with increases in these symptoms (depression, pain interference, pain catastrophizing, and cancer treatment distress). The study design was a randomized controlled crossover clinical trial comparing the VMWH-CBT intervention with education control. Participants (N\xa0=\xa044) received four sessions of both treatments, in a counterbalanced order (n\xa0=\xa022 per order condition). Participants were 89% female (N\xa0=\xa039) with mean age of 61\xa0years (SD

In [95]:
df_Abs.at[30,'AbL'] = 0

In [96]:
#df_Abs['Abstract'][31]: secondary analysis of noninvasive placebo acupuncture for insomnia

'To determine the patient characteristics that are associated with a response to noninvasive placebo acupuncture for insomnia. We performed a secondary analysis of three randomized, double-blind, placebo-controlled trials of acupuncture for primary insomnia and residual insomnia associated with major depression. A total of 86 participants were randomized to receive placebo acupuncture three times per week for three consecutive weeks. Outcome was assessed at 1-week posttreatment. Response was defined as an Insomnia Severity Index (ISI) score improved by eight points or more from baseline to 1-week posttreatment. Sociodemographic, clinical, and baseline characteristics including sleep diary-derived and actigraph-derived sleep parameters as predictors of placebo response were examined using univariate and multivariate logistic regression. The effect size of placebo acupuncture was estimated at 0.18 for total sleep time, 0.08 for sleep efficiency, and 0.92 for ISI score. Eighteen (20.9%) o

In [97]:
df_Abs.at[31,'AbL'] = 0

In [108]:
#df_Abs['Abstract'][42]:secondary analysis

'To conduct a secondary analysis of a randomized controlled trial (RCT) that aims to understand the mediating effects embedded in a mind-body-spirit intervention for sleep and mood disturbances. 126 adults with mild to moderate depression and subjective sleep disturbance, defined as Center for Epidemiologic Studies Depression Scale (CESD) scores from 10 to 34 and Pittsburgh Sleep Quality Index (PSQI) score\xa0>\xa05, participated in a waitlist-controlled RCT of an integrative mind-body-spirit intervention (I-BMS). Holistic well-being scale (HWS), a measure of the state of affliction and equanimity in mind, body and spirit, was included as a possible mediator. Data was collected at baseline and three-month follow-up. Mediation analyses were adopted to examine the pathways leading to sleep and mood improvements. After adjustments of baseline severities, changes in depressive symptoms partially mediated the effect of I-BMS on nighttime symptoms of insomnia (95% CI: 0.12-0.96), while exert

In [109]:
df_Abs.at[42,'AbL'] = 0

In [111]:
#df_Abs['Abstract'][44]: CBT plus IR

'Posttraumatic stress disorder (PTSD) is associated with cognitive deficits in attention, executive control, and memory, although few studies have investigated the relevance of cognitive difficulties for treatment outcomes. We examined whether cognitive functioning and history of traumatic brain injury (TBI) were associated with response to cognitive-behavioral therapy (CBT) for PTSD-related sleep problems. In a randomized controlled trial of Imagery Rehearsal (IR) added to components of CBT for Insomnia (IR\xa0+\xa0cCBT-I) compared to cCBT-I alone for PTSD-related recurrent nightmares, 94 U.S. veterans completed a battery of cognitive tests. TBI was assessed via structured clinical interview. Mixed-effects models examined main effects of cognitive functioning and interactions with time on primary sleep and nightmare outcomes. Significant verbal immediate memory by time interactions were found for nightmare distress, nightmare frequency, and sleep quality, even after controlling for ov

In [112]:
df_Abs.at[44,'AbL'] = 0

In [114]:
#df_Abs['Abstract'][46]:MBCT

'Many antidepressant medications (ADM) are associated with disruptions in sleep continuity that can compromise medication adherence and impede successful treatment. The present study investigated whether mindfulness meditation (MM) training could improve self-reported and objectively measured polysomnographic (PSG) sleep profiles in depressed individuals who had achieved at least partial remission with ADM, but still had residual sleep complaints. Twenty-three ADM users with sleep complaints were randomized into an 8-week Mindfulness-Based Cognitive Therapy (MBCT) course or a waitlist control condition. Pre-post measurements included PSG sleep studies and subjectively reported sleep, residual depression symptoms. Compared to controls, the MBCT participants improved on both PSG and subjective measures of sleep. They showed a pattern of decreased wake time and increased sleep efficiency. Sleep depth, as measured by stage 1 and slow-wave sleep, did not change as a result of mindfulness tr

In [115]:
df_Abs.at[46,'AbL'] = 0

In [119]:
#df_Abs['Abstract'][50]:CBT+music relaxiation

'The objective of the study was to evaluate the effects of cognitive behavioral therapy [sleep-related behavior modification and progressive muscle relaxation on insomnia of maintenance hemodialysis (MHD) patients] on improving insomnia of MHD patients. 103 MHD patients complicated with insomnia were randomly assigned to treatment (n = 52) and control (n = 51) groups. The control group was treated with conventional hemodialysis, and the treatment group was additionally treated with cognitive behavioral therapy for 3 months (sleep-related behavior modification and progressive muscle relaxation). All cases were assessed by Symptom Checklist 90 (SCL-90) and Pittsburgh Sleep Quality Index (PSQI) before and 2, 4, 6, 8, 10, and 12 weeks after treatment. Fifty-one patients in the treatment group and 47 patients in the control group completed the experiments. After treatment, the total mean scores were (1.94 ± 0.50/2.29 ± 0.31); scores of somatization, depression, anxiety, hostility, and addit

In [120]:
df_Abs.at[50,'AbL'] = 0

In [121]:
#df_Abs['Abstract'][51]:CBT-I with adjunctive IRT

'Sleep disturbances are a core and salient feature of posttraumatic stress disorder (PTSD). Pilot studies have indicated that combined cognitive-behavioral therapy for insomnia (CBT-I) and imagery rehearsal therapy (IRT) for nightmares improves sleep as well as PTSD symptoms. The present study randomized 40 combat veterans (mean age 37.7 years; 90% male and 60% African American) who served in Afghanistan and/or Iraq (Operation Enduring Freedom [OEF]/Operation Iraqi Freedom [OIF]) to 4 sessions of CBT-I with adjunctive IRT or a waitlist control group. Two thirds of participants had nightmares at least once per week and received the optional IRT module. At posttreatment, veterans who participated in CBT-I/IRT reported improved subjectively and objectively measured sleep, a reduction in PTSD symptom severity and PTSD-related nighttime symptoms, and a reduction in depression and distressed mood compared to the waitlist control group. The findings from this first controlled study with OEF/O

In [122]:
df_Abs.at[51,'AbL'] = 0

In [123]:
#df_Abs['Abstract'][52]: RCT of biofeedback timing 

"Clinical studies have shown that biofeedback-assisted relaxation positively influences the treatment outcomes of sleep disturbance. However, there are only few studies reporting the timing of relaxation training initiation, and the relationships between the timing of initiation and the effectiveness of relaxation remain unclear. The aim of this study was to determine the optimal timing for initiating nurse-led biofeedback-assisted relaxation on hospitalized coronary heart disease patients with sleep disturbance. An experimental pretest and repeated posttest design was used to compare the effectiveness of nurse-led biofeedback-assisted relaxation. A total of 128 patients with coronary heart disease were randomly assigned to 1 of 4 groups: morning group, night group, morning-night group, or control group. Outcome measures included self-report of sleep-related indicators, the scores of the Pittsburgh Sleep Quality Index (PSQI) and the Zung's Self-rating Anxiety Scale (SAS), and the dosag

In [124]:
df_Abs.at[52,'AbL'] = 0

In [128]:
#df_Abs['Abstract'][56]: CBT+mindfulness and adolescent

"The aim of this study was to test moderators of therapeutic improvement in an adolescent cognitive-behavioral and mindfulness-based group sleep intervention. Specifically, we examined whether the effects of the program on postintervention sleep outcomes were dependent on participant gender and/or measures of sleep duration, anxiety, depression, and self-efficacy prior to the interventions. Secondary analysis of a randomized controlled trial conducted with 123 adolescent participants (female\xa0=\xa059.34%; mean age\xa0=\xa014.48\xa0years, range 12.04-16.31\xa0years) who had elevated levels of sleep problems and anxiety symptoms. Participants were randomized into either a group sleep improvement intervention (n\xa0=\xa063) or group active control 'study skills' intervention (n\xa0=\xa060). The sleep intervention ('Sleep SENSE') was cognitive behavioral in approach, incorporating sleep education, sleep hygiene, stimulus control, and cognitive restructuring, but also had added anxiety-re

In [129]:
df_Abs.at[56,'AbL'] = 0

In [134]:
#df_Abs['Abstract'][61]:risk factor of acupuncture

"Mild adverse events (AEs) are common with acupuncture, but the risk factors remain unclear. A prospective study using a standardised AE assessment and acupuncture protocol was undertaken to address the question. A 20-item AE report form investigated local and systemic AEs in 150 adults with insomnia randomised to receive traditional, minimal and non-invasive sham acupuncture. Sociodemographic, clinical and psychological variables at baseline and past history and perceived credibility of acupuncture were assessed. The incidence of any AEs per patient was 42.4% with traditional acupuncture, 40.7% with minimal acupuncture and 16.7% with non-invasive sham acupuncture. Traditional and minimal acupuncture were associated with a greater number of local AEs, while the presence of a chronic medical condition was predictive of fewer local and systemic AEs. Greater severity of insomnia, anxiety, depression, somatic symptoms and pain catastrophising thoughts were associated with lower risk, but m

In [135]:
df_Abs.at[61,'AbL'] = 0

In [137]:
#df_Abs['Abstract'][63]: adolescents and CBT+mindfulness

'Sleep problems are a major risk factor for the emergence of depression in adolescence. The aim of this study was to test whether an intervention for improving sleep habits could prevent the emergence of depression, and improve well-being and cardiovascular indices amongst at-risk adolescents. A longitudinal randomised controlled trial (RCT) is being conducted across Victorian Secondary Schools in Melbourne, Australia. Adolescents (aged 12-17 years) were defined as at-risk for depression if they reported high levels of anxiety and sleep problems on in-school screening questionnaires and had no prior history of depression (assessed by clinical diagnostic interview). Eligible participants were randomised into either a sleep improvement intervention (based on cognitive behavioral and mindfulness principles) or an active control condition teaching study skills. Both programs consisted of seven 90 minute-long sessions over seven weeks. All participants were required to complete a battery of

In [138]:
df_Abs.at[63,'AbL'] = 0

In [145]:
#df_Abs['Abstract'][70]: CBT

"This study is one of the first randomized controlled trials investigating cognitive behavioral therapy for insomnia (CBT-I) delivered by a fully automated mobile phone app. Such an app can potentially increase the accessibility of insomnia treatment for the 10% of people who have insomnia. The objective of our study was to investigate the efficacy of CBT-I delivered via the Sleepcare mobile phone app, compared with a waitlist control group, in a randomized controlled trial. We recruited participants in the Netherlands with relatively mild insomnia disorder. After answering an online pretest questionnaire, they were randomly assigned to the app (n=74) or the waitlist condition (n=77). The app packaged a sleep diary, a relaxation exercise, sleep restriction exercise, and sleep hygiene and education. The app was fully automated and adjusted itself to a participant's progress. Program duration was 6 to 7 weeks, after which participants received posttest measurements and a 3-month follow-u

In [147]:
df_Abs.at[70,'AbL'] = 0

In [149]:
#df_Abs['Abstract'][71]: review

'"Yoga for improving health-related quality of life, mental health and cancer-related symptoms in women diagnosed with breast cancer."'

In [150]:
df_Abs.at[71,'AbL'] = 0

In [154]:
#df_Abs['Abstract'][75]: review

'This study aims at concluding the current evidence on the therapeutic effects of acupoints stimulation for cancer patients with anxiety and depression. Randomized controlled trials using acupoints stimulation for relieving anxiety and/or depression in cancer patients were searched, and 11 studies were finally included, of which eight trials compared acupoints stimulation with standard methods of treatment/care, and acupoints stimulation showed significantly better effects in improving depression than using standard methods of treatment/care. Four studies compared true acupoints stimulation with sham methods, and no significant differences can be found between groups for either depression or anxiety, although the pooled effects still favored true intervention. For the five studies that evaluated sleep quality, the results were conflicting, with three supporting the superiority of acupoints stimulation in improving sleep quality and two demonstrating no differences across groups. Acupoi

In [155]:
df_Abs.at[75,'AbL'] = 0

In [156]:
#df_Abs['Abstract'][76] in addition to anti-depression medicine

'To verify the anti-depression effect of acupuncture and moxibustion based on the medication with selective serotonin reuptake inhibitors (SSRIs). Eighty cases of depression were randomly divided into an acupuncture-moxibustion-medication group (25 cases), an acupuncture-medication group (25 cases) and a medication group (30 cases). SSRIs medication was administered in all of the three groups. Complementarily, in acupuncture-moxibustion-medication group, the needling technique of qi conduction in the Governor Vessel was applied to Baihui (GV 20), Fengfu (GV 16), Dazhui (GV 14), etc. Additionally, mild moxibustion was added at Dazhui (GV 14) and Baihui (GV 20). In acupuncture-medication group, acupuncture for qi conduction in the Governor Vessel was only adopted. Hamilton Depression Scale (HAMD) was used for the evaluation of the total score, the score of each factor before and after treatment separately, and the therapeutic effects were observed among 3 groups. Compared with medication

In [157]:
df_Abs.at[76,'AbL'] = 0

In [158]:
#df_Abs['Abstract'][77]: protocol

'Caregiving can be stressful, potentially creating physical and psychological strain. Substantial evidence has shown that family caregivers suffer from significant health problems arising from the demands of caregiving. Although there are programs supporting caregivers, there is little evidence regarding their effectiveness. Acupressure is an ancient Chinese healing method designed to restore the flow of Qi (vital energy) by applying external pressure to acupoints. A randomized, wait-list controlled trial was developed to evaluate the effectiveness of a self-administered acupressure intervention on caregiver stress (primary objective) and stress-related symptoms of fatigue, insomnia, depression, and health-related quality of life (secondary objectives) in Chinese caregivers of older family members. Two hundred Chinese participants, aged\u2009≥\u200921\xa0years, who are the primary caregivers of an older family member and screen positive for caregiver stress and symptoms of fatigue/inso

In [159]:
df_Abs.at[77,'AbL'] = 0

In [162]:
#df_Abs['Abstract'][80]: summarise

'Yoga is a mind-body practice that encompasses a system of postures (asana), deep breathing (pranayama), and meditation. Over 36 million Americans practice yoga of which the majority are reproductive-aged women. Literature to support this practice is limited, albeit on the rise. A prenatal yoga practice has been shown to benefit women who suffer from anxiety, depression, stress, low back pain, and sleep disturbances. A small number of studies have been performed in high-risk pregnancies that also demonstrate an improvement in outcomes. The safety of performing yoga for the first time in pregnancy and fetal tolerance has been demonstrated. '

In [163]:
df_Abs.at[80,'AbL'] = 0

In [164]:
#df_Abs['Abstract'][81] not CAM

'This paper describes a randomized controlled single blind study testing the effects of a patient education intervention combined with positive therapeutic suggestions on anxiety for cataract surgery patients. 84 patients participated in the study. Physiological and behavioral indicators of anxiety were compared between a regularly treated control and an intervention group receiving an audio CD containing information, relaxation, and positive imagery. We found that the intervention group was calmer throughout the four measurement points of the study (p=.004; d=0.71) and they were more cooperative (p=.01; d=0.60) during the operation. The groups did not differ in sleep quality before the day of the operation, heart rate during the procedure, and subjective Well-being. Findings indicate that preoperative information combined with positive suggestions and anxiety management techniques might reduce patient anxiety in the perioperative period of cataract surgery, but further research is nee

In [165]:
df_Abs.at[81,'AbL'] = 0

In [167]:
#df_Abs['Abstract'][83]: physical therapies

'Bruxism is a parafunctional habit characterized by grinding and/or clenching of the teeth. It may happen while awake (awake bruxism) or while sleeping (sleep bruxism). In adults, the prevalence is 20% for the awake bruxism and 8% for the sleep bruxism. Peripheral, central, and psychosocial factors influence the disorder, which may predispose to pain in the masticatory muscles and neck, headache, decreased pain thresholds in the masticatory and cervical muscles, limitation mandibular range of motion, sleep disorders, stress, anxiety, depression, and overall impairment of oral health. The aim of this study is to compare two distinct physical therapy interventions with dental treatment in pain, mandibular range of motion, sleep quality, anxiety, stress, depression, and oral health in individuals with bruxism. Participants will be randomized into one of three groups: Group 1 (n=24) intervention will consist of massage and stretching exercises; Group 2 (n=24) will consist of relaxation and

In [168]:
df_Abs.at[83,'AbL'] = 0

In [169]:
#df_Abs['Abstract'][84]:MBCT

"To capture any sleep quality changes associated with group psychotherapy. Physician-referred, chronically depressed patients (n\u2009=\u200925) were randomized to either eight group sessions of Mindfulness-based Cognitive Therapy (MBCT, n\u2009=\u20099) plus Treatment As Usual (TAU), or the Cognitive Behavioral Analysis System of Psychotherapy (CBASP, n\u2009=\u20098) plus TAU, or to TAU only (control group, n\u2009=\u20098). Participants recorded their sleep at home. The primary outcome variables were: stable and unstable sleep, which were assessed using cardiopulmonary coupling (CPC) analysis, and estimated total sleep and wake time (minutes). Cardiopulmonary coupling measures heart rate variability and the electrocardiogram's R-wave amplitude fluctuations associated with respiration. By post-treatment night 6, the CBASP group had more stable sleep (p=\u20090.044) and less wake (p\u2009=\u20090.004) compared with TAU, and less wake vs MBCT (p\u2009=\u20090.039). The CBASP group psyc

In [170]:
df_Abs.at[83,'AbL'] = 0

In [171]:
#df_Abs['Abstract'][84]: MBCT

"To capture any sleep quality changes associated with group psychotherapy. Physician-referred, chronically depressed patients (n\u2009=\u200925) were randomized to either eight group sessions of Mindfulness-based Cognitive Therapy (MBCT, n\u2009=\u20099) plus Treatment As Usual (TAU), or the Cognitive Behavioral Analysis System of Psychotherapy (CBASP, n\u2009=\u20098) plus TAU, or to TAU only (control group, n\u2009=\u20098). Participants recorded their sleep at home. The primary outcome variables were: stable and unstable sleep, which were assessed using cardiopulmonary coupling (CPC) analysis, and estimated total sleep and wake time (minutes). Cardiopulmonary coupling measures heart rate variability and the electrocardiogram's R-wave amplitude fluctuations associated with respiration. By post-treatment night 6, the CBASP group had more stable sleep (p=\u20090.044) and less wake (p\u2009=\u20090.004) compared with TAU, and less wake vs MBCT (p\u2009=\u20090.039). The CBASP group psyc

In [172]:
df_Abs.at[84,'AbL'] = 0

In [173]:
#df_Abs['Abstract'][85]: Protocol

'Hot flushes and night sweats (vasomotor symptoms) are common menopausal symptoms, often causing distress, sleep deprivation and reduced quality of life. Although hormone replacement therapy is an effective treatment, there are concerns about serious adverse events. Non-hormonal pharmacological therapies are less effective and can also cause adverse effects. Complementary therapies, including acupuncture, are commonly used for menopausal vasomotor symptoms. While the evidence for the effectiveness of acupuncture in treating vasomotor symptoms is inconclusive, acupuncture has a low risk of adverse effects, and two small studies suggest it may be more effective than non-insertive sham acupuncture. Our objective is to assess the efficacy of needle acupuncture in improving hot flush severity and frequency in menopausal women. Our current study design is informed by methods tested in a pilot study. This is a stratified, parallel, randomised sham-controlled trial with equal allocation of par

In [174]:
df_Abs.at[85,'AbL'] = 0

In [176]:
#df_Abs['Abstract'][87]: qulitative interview

'Non-pharmacological interventions for depression may help patients manage their condition. Evidence from a recent large-scale trial (ACUDep) suggests that acupuncture and counselling can provide longer-term benefits for many patients with depression. This paper describes the strategies practitioners reported using to promote longer-term benefits for their patients. A qualitative sub-study of practitioners (acupuncturists and counsellors) embedded in a randomised controlled trial. Using topic guides, data was collected from telephone interviews and a focus group, altogether involving 19 counsellors and 17 acupuncturists. Data were audio recorded, transcribed verbatim and analysed using thematic content analysis. For longer-term impact, both acupuncturists and counsellors encouraged insight into root causes of depression on an individual basis and saw small incremental changes as precursors to sustained benefit. Acupuncturists stressed the importance of addressing concurrent physical sy

In [177]:
df_Abs.at[87,'AbL'] = 0

In [178]:
#df_Abs['Abstract'][88]:protocol

'<i>Background</i>. Depression is frequently accompanied by sleep disturbances including insomnia. Insomnia may persist even after mood symptoms have been adequately treated. Acupuncture is considered to be beneficial to adjust the state of body and mind and restore the normal sleep-awake cycle. This trial is aimed at evaluating the efficacy and safety of electroacupuncture on treating insomnia in patients with depression. <i>Methods</i>. We describe a protocol for a randomized, single-blinded, sham controlled trial. Ninety eligible patients will be randomly assigned to one of 3 treatment groups: treatment group (acupuncture), control A group (superficial acupuncture at sham points), and control B group (sham acupuncture). All treatment will be given 3 times per week for 8 weeks. The primary outcome is the Pittsburgh Sleep Quality Index (PSQI). The secondary outcomes are sleep parameters recorded in the Actigraphy, Hamilton Rating Scale for Depression (HAMD), and Self-Rating Depression

In [179]:
df_Abs.at[88,'AbL'] = 0

In [180]:
#df_Abs['Abstract'][89]: school boy; adolescent

'Mindfulness-based stress reduction (MBSR) has been shown to improve mental health and reduce stress in a variety of adult populations. Here, we explore the effects of a school-based MBSR program for young urban males. In fall 2009, 7th and 8th graders at a small school for low-income urban boys were randomly assigned to 12-session programs of MBSR or health education (Healthy Topics-HT). Data were collected at baseline, post-program, and three-month follow-up on psychological functioning; sleep; and salivary cortisol, a physiologic measure of stress. Forty-one (22 MBSR and 19 HT) of the 42 eligible boys participated, of whom 95% were African American, with a mean age of 12.5 years. Following the programs, MBSR boys had less anxiety (p=0.01), less rumination (p=0.02), and showed a trend for less negative coping (p=0.06) than HT boys. Comparing baseline with post-program, cortisol levels increased during the academic terms for HT participants at a trend level (p=0.07) but remained const

In [181]:
df_Abs.at[89,'AbL'] = 0

In [182]:
#df_Abs['Abstract'][90]: hypnosis plus CPT

'Despite the success of empirically supported treatments for posttraumatic stress disorder (PTSD), sleep impairment frequently remains refractory after treatment. This single-site, randomized controlled trial examined the effectiveness of sleep-directed hypnosis as a complement to an empirically supported psychotherapy for PTSD (cognitive processing therapy [CPT]). Participants completed either 3 weeks of hypnosis (n = 52) or a symptom monitoring control condition (n = 56) before beginning standard CPT. Multilevel modeling was used to investigate differential patterns of change to determine whether hypnosis resulted in improvements in sleep, PTSD, and depression. An intervening variable approach was then used to determine whether improvements in sleep achieved during hypnosis augmented change in PTSD and depression during CPT. After the initial phase of treatment (hypnosis or symptom monitoring), the hypnosis condition showed significantly greater improvement than the control condition

In [183]:
df_Abs.at[90,'AbL'] = 0

In [184]:
#df_Abs['Abstract'][91]: protocol

'Mindfulness-based interventions, modified and shortened versions of meditation teachings, have proved to be effective in the improvement of quality of life in many clinical conditions, including chronic diseases. Preliminary results available in the literature and in clinical experience indicate a high potential for this treatment for the reduction of psychological suffering in people with chronic diseases. This randomized controlled trial will investigate the impact of a multiple sclerosis (MS) specific telemedicine meditation intervention on the quality of life of people with multiple sclerosis and their caregivers. This trial will recruit 120 patients, men and women, with a diagnosis of relapsing-remitting or secondary progressive MS and their caregivers to participate in a 2-month intervention. Patients will undergo assessments of quality of life, anxiety, depression, quality of sleep, mindfulness and fatigue levels conducted at baseline, at week 8 (conclusion of the intervention)

In [185]:
df_Abs.at[91,'AbL'] = 0

In [187]:
#df_Abs['Abstract'][93]： adolescent

"Sleep problems are common and persistent during adolescence and can have negative effects on adolescents' mood. To date, studies that investigate the effects of sleep extension on adolescents' sleep and depressive symptoms are still lacking. This study aims to investigate the effects of gradual sleep extension combined with sleep hygiene advice in adolescents with chronic sleep reduction on objectively measured sleep, self-reported sleep problems and depressive symptoms. Fifty-five adolescents with chronic sleep reduction (mean age: 15.44\xa0years; 85.5% females) were included in the study. Participants were randomly assigned to either a sleep extension group (gradual sleep extension by advancing bedtimes in the evening and receiving sleep hygiene advice) or to a control group (no instruction). Sleep was measured with actigraphy during three\xa0weeks, the first week was the baseline week, and the last two\xa0weeks were the experimental weeks during which sleep was extended. Other outc

In [188]:
df_Abs.at[93,'AbL'] = 0

In [190]:
#df_Abs['Abstract'][94]: protocol

'It is reported that nursing is one of the most vulnerable jobs for developing depression. While they may not be clinically diagnosed as depressed, nurses often suffer from depression and anxiety symptoms, which can lead to a low level of patient care. However, there is no rigorous evidence base for determining an effective prevention strategy for these symptoms in nurses. After reviewing previous literature, we chose a strategy of treatment with omega-3 fatty acids and a mindfulness-based stress management program for this purpose. We aim to explore the effectiveness of these intervention options for junior nurses working in hospital wards in Japan. A factorial-design multi-center randomized trial is currently being conducted. A total of 120 nurses without a managerial position, who work for general hospitals and gave informed consent, have been randomly allocated to a stress management program or psychoeducation using a leaflet, and to omega-3 fatty acids or identical placebo pills. 

In [191]:
df_Abs.at[94,'AbL'] = 0

In [192]:
#df_Abs['Abstract'][95]：review

"Shiatsu, similar to acupressure, uses finger pressure, manipulations and stretches, along Traditional Chinese Medicine meridians. Shiatsu is popular in Europe, but lacks reviews on its evidence-base. Acupressure and Shiatsu clinical trials were identified using the MeSH term 'acupressure' in: EBM reviews; AMED; BNI; CINAHL; EMBASE; MEDLINE; PsycARTICLES; Science Direct; Blackwell Synergy; Ingenta Select; Wiley Interscience; Index to Theses and ZETOC. References of articles were checked. Inclusion criteria were Shiatsu or acupressure administered manually/bodily, published after January 1990. Two reviewers performed independent study selection and evaluation of study design and reporting, using standardised checklists (CONSORT, TREND, CASP and STRICTA). Searches identified 1714 publications. Final inclusions were 9 Shiatsu and 71 acupressure studies. A quarter were graded A (highest quality). Shiatsu studies comprised 1 RCT, three controlled non-randomised, one within-subjects, one obs

In [193]:
df_Abs.at[95,'AbL'] = 0

In [195]:
#df_Abs['Abstract'][97]:protocol

'Individuals living with cancer must learn to face not only the physical symptoms of their condition, but also the anxiety and uncertainty related to the progression of the disease, the anticipation of physical and emotional pain related to illness and treatment, the significant changes implied in living with cancer, as well as the fear of recurrence after remission. Mindfulness-based meditation constitutes a promising option to alleviate these manifestations. This article presents the rationale and protocol development for a research project aimed at evaluating the effects of a mindfulness-based meditation intervention on quality of life, sleep, and mood in adolescents with cancer compared to a control group. A prospective, longitudinal, experimental design involving three time points (baseline, post-intervention, and follow-up) and two groups (experimental and control) was developed for this project. Participants will be assigned randomly to either group. Eligible participants are ad

In [196]:
df_Abs.at[97,'AbL'] = 0

In [197]:
#df_Abs['Abstract'][98]:review

"The goal was to review systematically the comparative effectiveness of yoga, compared with other exercise interventions, for older adults as shown on measures of health and physical functioning. This was a systematic review with both narrative synthesis and meta-analysis. Searches were conducted in MEDLINE®/PUBMED, PSYCINFO, CINAHL, Web of Science, and SCOPUS; bibliographies of selected articles; and one systematic review on the effects of yoga on cardiovascular disease. Original studies from 1950 to November 2010 were sought, evaluating the effects of yoga on older adults. The search was restricted to randomized controlled trials of yoga in subjects ≥age 60, and published in English. Data were extracted and evaluated regarding setting, population size and characteristics, intervention type and duration, comparison group, outcome assessment, data analysis, follow-up, key results, and the quality of each study according to specific predetermined criteria. The search yielded 18 eligible

In [198]:
df_Abs.at[98,'AbL'] = 0

In [199]:
#df_Abs['Abstract'][99]: secondary analysis of RCT

"Fatigue is not only a core symptom of major depressive disorder (MDD), but also a common residual symptom. We determined the sociodemographic, clinical, and pharmacologic factors that were associated with fatigue in patients with remission or partial remission of MDD. Data was derived from a randomized controlled trial of acupuncture in 137 MDD patients with residual symptoms. Fatigue was measured by Multidimensional Fatigue Inventory (MFI-20). Self-report and clinician-rated scales were used to assess psychopathology. 17-item Hamilton Depression Rating Scale (HDRS17) score≤7 denoted MDD remission. Participants' average HDRS17 score was 10.5; 29.2% were in remission. The average MFI-20 score was 71.8; 83.2% had severe fatigue, defined as MFI-20 score≥60. Fifty-two of 137 participants (38%) were using sedating psychotropic medications. Antidepressant dosage ranged from 1-90mg fluoxetine equivalent and sedatives/hypnotics from 1-60mg diazepam equivalent. There were significant correlati

In [200]:
df_Abs.at[99,'AbL'] = 0

In [201]:
#df_Abs['Abstract'][100]: not CAM

'The aim of this study was to conduct a preliminary dismantling study of exposure, relaxation, and rescripting therapy (ERRT) for nightmares. Participants (N = 70) were randomized to 1 of 2 conditions: ERRT with nightmare exposure and rescripting (full protocol; EX) or ERRT without nightmare exposure and rescripting (NEX). Participants completed follow-ups at 1 week, 3 months, and 6 months posttreatment. Both conditions yielded statistically significant improvements with medium to large effect sizes on primary outcomes related to nightmares (i.e., nights with nightmares, nightmares per week, and nightmare severity). Both conditions also yielded statistically significant improvements on secondary outcomes with medium to large effect sizes on fear of sleep, sleep quality, insomnia severity, daytime sleepiness, posttraumatic stress disorder symptom severity, and depression severity. The conditions did not differ at any time point. Findings indicate that ERRT with and without nightmare exp

In [202]:
df_Abs.at[100,'AbL'] = 0

In [203]:
#df_Abs['Abstract'][101]: secondary analysis of RCT

"Sleep disturbances and depressive symptoms are associated closely with daytime dysfunctions, yet few studies have investigated their temporal relationship in a randomized controlled trial. We investigated the inter-relationships among sleep, depressive symptoms and daytime functioning following an integrative body-mind-spirit (I-BMS) intervention. One hundred and eighty-five participants (mean age 55.28\xa0years, 75.1% female) with co-existing sleep and depressive symptoms were randomized to I-BMS or waiting-list. Daytime functioning variables included the daytime dysfunction items of the Pittsburg Sleep Quality Index (PSQI-day), Somatic Symptom Inventory, Hospital Anxiety Depression Scale and Short Form Health Survey collected at baseline, post-treatment and 3-month follow-up. Sleep and depressive symptoms were measured by the sleep items of the PSQI (PSQI-night) and Center for Epidemiological Studies Depression Scale (excluding the sleep item) (CESD-M). Regression and path analyses 

In [204]:
df_Abs.at[101,'AbL'] = 0

In [205]:
#df_Abs['Abstract'][102]: CBT plus catgut implantation

'This study was conducted to evaluate the effectiveness of a cognitive behavioral self-help program (SHP) in combination with pharmacotherapy in patients with primary insomnia in general practice. Patients were recruited from 31 general practitioners (GPs) in the Hamburg area, who were randomly assigned to the two different study conditions. Eighty patients completed the study. They had suffered from insomnia for several years and showed a high impairment according to the Pittsburgh Sleep Quality Index. According to assignment of their GP the patients either received a progressively reduced 4-week pharmacotherapy or a combination of pharmacotherapy and a SHP consisting of six chapters on progressive muscle relaxation, cognitive relaxation, modified stimulus control, thought stopping, and cognitive restructuring. After study enrollment patients had short weekly consultations with their GPs during treatment to receive sleep medication and questionnaires. They completed questionnaires mea

In [206]:
df_Abs.at[102,'AbL'] = 0

In [210]:
#df_Abs['Abstract'][106]: review

"Malignant neoplasms of the lymphoid or myeloid cell lines including lymphoma, leukaemia and myeloma are referred to as haematological malignancies. Complementary and alternative treatment options such as meditation practice or yoga are becoming popular by treating all aspects of the disease including physical and psychological symptoms. However, there is still unclear evidence about meditation's effectiveness, and how its practice affects the lives of haematologically-diseased patients. This review aims to assess the benefits and harms of meditation practice as an additional treatment to standard care for adults with haematological malignancies. We searched the Cochrane Central Register of Controlled Trials (CENTRAL, Issue 8, 2015), MEDLINE (1950 to August 2015), databases of ongoing trials, the metaRegister of Controlled Trials (mRCT) (http://www.controlled-trials.com/mrct/), conference proceedings of annual meetings of: the American Society of Hematology; American Society of Clinica

In [211]:
df_Abs.at[106,'AbL'] = 0

In [212]:
#df_Abs['Abstract'][107]: protocol 

'Conventional approaches for benzodiazepine tapering have their limitations. Anecdotal studies have shown that acupuncture is a potential treatment for facilitating successful benzodiazepine tapering. As of today, there was no randomized controlled trial examining its efficacy and safety. The purpose of the study is to evaluate the efficacy of using electroacupuncture as an adjunct treatment to gradual tapering of benzodiazepine doses in complete benzodiazepine cessation in long-term benzodiazepine users. The study protocol of a randomized, assessor- and subject-blinded, controlled trial is presented. One hundred and forty-four patients with histories of using benzodiazepines in ≥50% of days for more than 3\xa0months will be randomly assigned in a 1:1 ratio to receive either electroacupuncture or placebo electroacupuncture combined with gradual benzodiazepine tapering schedule. Both experimental and placebo treatments will be delivered twice per week for 4\xa0weeks. Major assessments w

In [213]:
df_Abs.at[107,'AbL'] = 0

In [214]:
#df_Abs['Abstract'][108]: proposed trial

'Clinically significant psychological distress in pregnancy is common, with epidemiological research suggesting that between 15 and 25\xa0% of pregnant women experience elevated symptoms of stress, anxiety, and depression. Untreated psychological distress in pregnancy is associated with poor obstetrical outcomes, changes in maternal physiology, elevated incidence of child physical and psychological disorders, and is predictive of maternal postpartum mood disorders. Despite the wide-ranging impact of antenatal psychological distress on mothers and their children, there is a gap in our knowledge about the most effective treatments that are available for psychological distress experienced in pregnancy. Additionally, no trials have focused on potential physiological changes that may occur as a result of receiving mindfulness training in pregnancy. The proposed trial will determine the effectiveness of an 8-week modified Mindfulness-based Cognitive Therapy (MBCT) intervention delivered duri

In [215]:
df_Abs.at[108,'AbL'] = 0

In [222]:
#df_Abs['Abstract'][115]: review

'This systematic review assessed clinical evidence for the use of acupuncture as an add-on treatment in patients with depression and schizophrenia and for its underlying working mechanisms. Four databases (Medline, Scopus, ERIC, and the Cochrane Library) were searched with a cutoff date of March 31, 2014. Systematic reviews and meta-analyses of acupuncture treatment for depression and schizophrenia were considered for inclusion. The scarcity of acupuncture research involving schizophrenia led to the inclusion of randomized controlled trials and case studies. The primary and secondary aims of this study were to evaluate the effects of acupuncture in treating patients with depression or schizophrenia and the possible working mechanisms underlying acupuncture through a systematic literature review. The overall clinical results on using acupuncture to treat depression are promising, but only limited evidence for its effectiveness in treating schizophrenia was found. Acupuncture improves th

In [223]:
df_Abs.at[115,'AbL'] = 0

In [225]:
#df_Abs['Abstract'][117]:protocol

"Hot flashes are a highly prevalent problem associated with menopause and breast cancer treatments. The recent findings from the Women's Health Initiative have important implications for the significance of a non-hormonal, mind-body intervention for hot flashes in breast cancer survivors. Women who take hormone therapy long-term may have a 1.2 to 2.0 fold increased risk of developing breast cancer. In addition, it is now known that hormone therapy with estrogen and progestin is associated with increased risk of cardiovascular disease and stroke. Currently there are limited options to hormone replacement therapy as non-hormonal pharmacological agents are associated with only modest activity and many adverse side effects. Because of this there is a need for more alternative, non-hormonal therapies. Hypnosis is a mind-body intervention that has been shown to reduce self-reported hot flashes by up to 68% among breast cancer survivors, however, the use of hypnosis for hot flashes among post

In [226]:
df_Abs.at[117,'AbL'] = 0

In [227]:
#df_Abs['Abstract'][118]:protocol

'Phantom limb pain is a prevalent condition that is difficult to manage, with a lack of robust evidence to support the use of many adjunctive treatments. Acupuncture can be effective in the management of many painful conditions but little is known about its effectiveness in treating phantom limb pain. The aim of this study is to explore the feasibility of conducting a randomized controlled trial comparing acupuncture and routine care in a group of lower limb amputees with phantom limb pain. An unstratified, pragmatic, randomized, two-armed, controlled trial of parallel design comparing acupuncture and usual care control will be conducted. A total of 20 participants will be randomly assigned to receive either usual care or usual care plus acupuncture. Acupuncture will include eight 1 hour treatments delivered pragmatically over 4 weeks by practitioners trained in traditional Chinese medicine. As outcome measures, the Numerical Pain Rating Scale, short-form McGill Pain Questionnaire 2, E

In [228]:
df_Abs.at[118,'AbL'] = 0

In [232]:
#df_Abs['Abstract'][122]: qualitative interview

'There is increasing recognition of mindfulness and mindfulness training as a way to decrease stress and increase psychological functioning. The aim of this study was to examine the perceived effects and experiences of mindfulness stress-reduction training as described by patients with cancer participating in a mindfulness-based stress-reduction training program. The study is based on analysis of data collected in semistructured interviews of the participants (n = 18) after the completion of a mindfulness-meditation training program. The intervention consisted of an 8-week mindfulness-training course. Most participants expressed a number of perceived positive effects of participating in the mindfulness program including increased calm, enhanced sleep quality, more energy, less physical pain, and increased well-being. However, a few participants experienced no effect. Both negative and positive views of the specific meditation and yoga exercises included in the program were expressed. C

In [233]:
df_Abs.at[122,'AbL'] = 0

In [234]:
#df_Abs['Abstract'][123]: review

'People living with chronic kidney disease (CKD) experience a range of symptoms and often have complex comorbidities. Many pharmacological interventions for people with CKD have known risks of adverse events. Acupuncture is widely used for symptom management in patients with chronic diseases and in other palliative care settings. However, the safety and efficacy of acupuncture for people with CKD remains largely unknown. We aimed to evaluate the benefits and harms of acupuncture, electro-acupuncture, acupressure, moxibustion and other acupuncture-related interventions (alone or combined with other acupuncture-related interventions) for symptoms of CKD. In particular, we planned to compare acupuncture and related interventions with conventional medicine, active non-pharmacological interventions, and routine care for symptoms of CKD. We searched the Cochrane Kidney and Transplant Specialised Register up to 28 January 2016 through contact with the Information Specialist using search terms

In [235]:
df_Abs.at[123,'AbL'] = 0

In [237]:
#df_Abs['Abstract'][125]: systematic review

"Interest in the application of yoga for health benefits in western medicine is growing rapidly, with a significant rise in publications. The purpose of this systematic review is to determine whether the inclusion of yoga therapy to the treatment of breast cancer can improve the patient's physical and psychosocial quality of life (QoL). A search of peer reviewed journal articles published between January 2009 and July 2014 was conducted. Studies were included if they had more than 15 study participants, included interventions such as mindfulness-based stress reduction (MBSR) or yoga therapy with or without comparison groups and had stated physical or psychological outcomes. Screening identified 38 appropriate articles. The most reported psychosocial benefits of yoga therapy were anxiety, emotional and social functioning, stress, depression and global QoL. The most reported physical benefits of yoga therapy were improved salivary cortisol readings, sleep quality and lymphocyte apoptosis

In [238]:
df_Abs.at[125,'AbL'] = 0

In [239]:
#df_Abs['Abstract'][126]: not CAM

'Nightmares are a common and serious problem in psychotherapeutic practice, although they are seldom considered as independent mental disorders. There are some promising approaches to the treatment of nightmares, notably Imagery Rehearsal Therapy, a cognitive-restructuring treatment. The core of this approach is the modification of the nightmare script and repeated imagination of the new script. However, most evaluation surveys have been conducted only with trauma patients, and thus far there is no standardized manual in the German language. 69 participants were examined using self-rating questionnaires. Participants belonged to three groups: 22 primarily nightmare sufferers, 21 patients with major depression and nightmares, 26 with PTSD and nightmares. 12 of the PTSD patients were randomly assigned to a control condition. Primary outcome measures were nightmare frequency and anxiety during nightmares. Overall, nightmare frequency and the anxiety they caused decreased following the tre

In [240]:
df_Abs.at[126,'AbL'] = 0

In [241]:
#df_Abs['Abstract'][127]: systematic review

"To quantify the effects of mindfulness-based therapy (MBT) on physical health, psychological health and quality of life (QOL) in patients with breast cancer. Studies were identified through a systematic search of six electronic databases. Randomized control trials (RCTs) examining the effects of MBT, versus a control group receiving no intervention on physical health, psychological health and QOL in breast cancer patients were included. Two authors independently assessed the methodological quality of included studies using a quality-scoring instrument developed by Jadad et al. and extracted relevant information according to a predesigned extraction form. Data was analysed using the Cochrane Collaboration's Revman5.1. Finally, seven studies involving 951 patients were included. While limited in power, the results of meta-analysis indicated a positive effect of MBT in reducing anxiety [SMD -0.31, 95% CI -0.46 to -0.16, P<0.0001], depression[SMD -1.13, 95% CI -1.85 to -0.41, P=0.002], fe

In [242]:
df_Abs.at[127,'AbL'] = 0

In [245]:
#df_Abs['Abstract'][130]: 'SYR 2013 Accepted Poster abstracts (title list)

'SYR 2013 Accepted Poster abstracts: 1. Benefits of Yoga as a Wellness Practice in a Veterans Affairs (VA) Health Care Setting: If You Build It, Will They Come? 2. Yoga-based Psychotherapy Group With Urban Youth Exposed to Trauma. 3. Embodied Health: The Effects of a Mind�Body Course for Medical Students. 4. Interoceptive Awareness and Vegetable Intake After a Yoga and Stress Management Intervention. 5. Yoga Reduces Performance Anxiety in Adolescent Musicians. 6. Designing and Implementing a Therapeutic Yoga Program for Older Women With Knee Osteoarthritis. 7. Yoga and Life Skills Eating Disorder Prevention Among 5th Grade Females: A Controlled Trial. 8. A Randomized, Controlled Trial Comparing the Impact of Yoga and Physical Education on the Emotional and Behavioral Functioning of Middle School Children. 9. Feasibility of a Multisite, Community based Randomized Study of Yoga and Wellness Education for Women With Breast Cancer Undergoing Chemotherapy. 10. A Delphi Study for the Develop

In [246]:
df_Abs.at[130,'AbL'] = 0

In [249]:
#df_Abs['Abstract'][133]: review

'Although some studies evaluated the effectiveness of massage therapy for fibromyalgia (FM), the role of massage therapy in the management of FM remained controversial. The purpose of this systematic review is to evaluate the evidence of massage therapy for patients with FM. Electronic databases (up to June 2013) were searched to identify relevant studies. The main outcome measures were pain, anxiety, depression, and sleep disturbance. Two reviewers independently abstracted data and appraised risk of bias. The risk of bias of eligible studies was assessed based on Cochrane tools. Standardised mean difference (SMD) and 95% confidence intervals (CI) were calculated by more conservative random-effects model. And heterogeneity was assessed based on the I(2) statistic. Nine randomized controlled trials involving 404 patients met the inclusion criteria. The meta-analyses showed that massage therapy with duration ≥ 5 weeks significantly improved pain (SMD, 0.62; 95% CI 0.05 to 1.20; p = 0.03)

In [250]:
df_Abs.at[133,'AbL'] = 0

In [252]:
#df_Abs['Abstract'][135]

'Post amputation, the complication of phantom limb pain (PLP) is prevalent and difficult to manage. This study aimed to determine whether it was feasible and acceptable to undertake a definitive multicentred randomised controlled trial assessing the effectiveness of acupuncture for treating lower limb amputees with PLP. A mixed-methods embedded design, including a randomised controlled trial and semistructured interviews, was undertaken. A total of 15 participants with PLP were randomly assigned to receive either eight pragmatic Traditional Chinese Medicine acupuncture treatments and usual care or usual care alone over 4\xa0weeks. Outcome measures were completed at baseline, weekly throughout the study and at 1\xa0month post completion of the study and included: a numerical pain-rating scale, the Short-Form McGill Pain Questionnaire 2, the EQ-5D-5\xa0L, the Hospital Anxiety and Depression Scale, the Perceived Stress Scale 10-item, the Insomnia Severity Index, and the Patient Global Imp

In [253]:
df_Abs.at[135,'AbL'] = 0

In [254]:
#df_Abs['Abstract'][136]: CBt and relaxiation

'This study aims to examine the effectiveness of a self-management multimodal comprehensive coping strategy program (CCSP) on quality of life (QOL) among breast cancer patients 1 year after treatment. Patients (n = 110) with stage II, III, or IV breast cancer scheduled to receive high dose chemotherapy and autologous hematopoietic stem cell transplantation were randomized to either CCSP treatment or control group. The CCSP intervention was taught 2 week before hospital admission with reinforcement at specified times during treatment and 3 months after discharge. The CCSP components included educational information, cognitive restructuring, coping skills enhancement, and relaxation with guided imagery. Instruments administered at baseline included the following: Quality of Life Index-Cancer Version (QOLI-CV), State-Trait Anxiety Inventory, Beck Depression Inventory, and Coping Strategies Questionnaire. At 1-year follow-up, patients (n = 73) completed and returned the follow-up QOLI-CV. 

In [257]:
df_Abs.at[136,'AbL'] = 0

In [259]:
#df_Abs['Abstract'][138]: not CAM

"Sleep disturbances are often referred to as a hallmark and as core symptoms of post-traumatic stress disorder (PTSD). Untreated sleep disturbances can contribute to the maintenance and exacerbation of PTSD symptoms, which may diminish treatment response and constitute a risk factor for poor treatment outcome. Controlled trials on treatment of sleep disturbances in refugees suffering from PTSD are scarce. The present study aims to examine sleep-enhancing treatment in refugees with PTSD. We aim to assess if add-on treatment with mianserin and/or Imagery Rehearsal Therapy (IRT) to treatment as usual (TAU) for PTSD improves sleep disturbances. We will study the relation between sleep disturbances, PTSD symptoms, psychosocial functioning and quality of life. The study is a randomised controlled superiority trial with a 2\u2009×\u20092 factorial design. The study will include 230 trauma-affected refugees. The patients are randomised into four groups. All four groups receive TAU - an interdi

In [261]:
df_Abs.at[138,'AbL'] = 0

In [263]:
#df_Abs['Abstract'][140]: not CAM

'Evidence supports the use of cognitive behavioral therapies for nightmares in trauma-exposed individuals. This randomized clinical trial replicated a study of exposure, relaxation, and rescripting therapy(ERRT) and extended prior research by including broad measures of mental health difficulties, self-reported physical health problems, and quality of life. Additionally, physiological correlates of treatment-related change assessed from a script-driven imagery paradigm were examined. Forty-seven individuals were randomized to treatment or waitlist control. The treatment group demonstrated improvements relative to the control group at the one-week post-treatment assessment. At the 6-month follow-up assessment, significant improvements were found for frequency and severity of nightmares, posttraumatic stress disorder symptoms, depression, sleep quality and quantity, physical health symptoms, anger, dissociation, and tension reduction behaviors. Participants also reported improved quality

In [264]:
df_Abs.at[140,'AbL'] = 0

In [265]:
#df_Abs['Abstract'][141]: systematic review

'The systematic review aimed to evaluate the effectiveness of massage in fibromyalgia. An electronic search was conducted at MEDLINE, SCiELO, EMBASE, ISI, PEDro, SPORTDiscus, CINAHL, Cochrane CENTRAL and LILACS (Jan 1990-May 2013). Ten randomized and non-randomized controlled trials investigating the effects of massage alone on symptoms and health-related quality of life of adult patients with fibromyalgia were included. Two reviewers independently screened records, examined full-text reports for compliance with the eligibility criteria, and extracted data. Meta-analysis (pooled from 145 participants) shows that myofascial release had large, positive effects on pain and medium effects on anxiety and depression at the end of treatment, in contrast with placebo; effects on pain and depression were maintained in the medium and short term, respectively. Narrative analysis suggests that: myofascial release also improves fatigue, stiffness and quality of life; connective tissue massage impro

In [266]:
df_Abs.at[141,'AbL'] = 0

In [267]:
#df_Abs['Abstract'][142]: anti-depressant, not CAM 

'This study evaluates the efficacy of agomelatine, the first antidepressant to be an agonist at MT(1)/MT(2) receptors and an antagonist at 5-HT(2C) receptors, versus sertraline with regard to the amplitude of the circadian rest-activity cycle and depressive and anxiety symptoms in patients with major depressive disorder (MDD). Outpatients with DSM-IV-TR-defined MDD received either agomelatine 25 to 50 mg (n = 154) or sertraline 50 to 100 mg (n = 159) during a 6-week, randomized, double-blind treatment period. The study was conducted from 2005 to 2006. The main outcome measure was the relative amplitude of the individual rest-activity cycles, expressed as change from baseline to week 6 and collected from continuous records using wrist actigraphy and sleep logs. Secondary outcome measures were sleep efficiency and sleep latency, both derived from actigraphy, and efficacy on depression symptoms (17-Item Hamilton Depression Rating Scale total score and Clinical Global Impressions scale sco

In [268]:
df_Abs.at[142,'AbL'] = 0

In [271]:
#df_Abs['Abstract'][145] : review

'Chronic pain and its associated distress and disability are common reasons for seeking medical help. Patients with chronic pain use primary healthcare services five times more than the rest of the population. Mindfulness has become an increasingly popular self-management technique. To assess the effectiveness of mindfulness-based interventions for patients with chronic pain. Systematic review and meta-analysis including randomised controlled trials of mindfulness-based interventions for chronic pain. There was no restriction to study site or setting. The databases MEDLINE(®), Embase, AMED, CINAHL, PsycINFO, and Index to Theses were searched. Titles, abstracts, and full texts were screened iteratively against inclusion criteria of: randomised controlled trials of mindfulness-based intervention; patients with non-malignant chronic pain; and economic, clinical, or humanistic outcome reported. Included studies were assessed with the Yates Quality Rating Scale. Meta-analysis was conducted.

In [272]:
df_Abs.at[145,'AbL'] = 0

In [275]:
#df_Abs['Abstract'][148] : hypnosis combine CBT

'Attitudes about hypnosis are associated with hypnotic responsiveness. However, little is known about how hypnosis attitudes change with treatment and if those changes are associated with better outcomes. This study examined whether an intervention based on the Valencia Model of Waking Hypnosis combined with Cognitive-Behavioral Therapy changed attitudes about hypnosis in a sample of patients with a history of cancer. The results indicated that the intervention improved attitudes toward hypnosis, relative to a control intervention, and the improvements remained stable at 3-month follow-up. Analyses also showed that changes in some attitudes were associated with treatment-related improvements. The findings are consistent with the idea that attitudes about hypnosis play a role in hypnosis treatment outcome, supporting the importance of addressing such beliefs at the onset of and throughout treatment.'

In [276]:
df_Abs.at[148,'AbL'] = 0

In [11]:
#df_Abs['Abstract'][151]: not CAM

'Research has shown that those with insomnia focus primarily on their sleep as a cause of daytime fatigue rather than the multitude of other possible causes of fatigue. This can create sleep-related anxiety and further perpetuate the sleep disturbance. In order to lessen the increased focus on sleep, the present study investigated whether people could learn to consider other attributions for fatigue via an information-based manipulation. Undergraduate students (N = 88) were randomized to two information groups: They either received information about common factors that could explain daytime fatigue (the fatigue information condition) or received generic sleep-related information (the control condition). Each group was tested pre- and post-intervention. Fatigue information participants were significantly more likely to consider non-sleep-related attributions for fatigue at post-intervention, relative to control participants. These results demonstrate that attributions for fatigue may be

In [85]:
df_Abs.at[151,'AbL'] = 0

In [86]:
#df_Abs['Abstract'][152]: meta-analysis

In [87]:
df_Abs.at[152,'AbL'] = 0

In [88]:
#df_Abs['Abstract'][154]: CBT + relaxation

In [89]:
df_Abs.at[154,'AbL'] = 0

In [90]:
#df_Abs['Abstract'][155]: participant of children

In [91]:
df_Abs.at[155,'AbL'] = 0

In [92]:
#df_Abs['Abstract'][156]: systematic review

In [93]:
df_Abs.at[156,'AbL'] = 0

In [94]:
#df_Abs['Abstract'][157]:systematic review

In [95]:
df_Abs.at[157,'AbL'] = 0

In [96]:
#df_Abs['Abstract'][158]: protocol

In [97]:
df_Abs.at[158,'AbL'] = 0

In [98]:
#df_Abs['Abstract'][161]: not RCT

In [99]:
df_Abs.at[161,'AbL'] = 0

In [100]:
#df_Abs['Abstract'][164]: acupuncture combined with seroxat

In [101]:
df_Abs.at[164,'AbL'] = 0

In [102]:
#df_Abs['Abstract'][165]: not RCT

In [103]:
df_Abs.at[165,'AbL'] = 0

In [104]:
#df_Abs['Abstract'][166]: pilot for pharmacotherapy RCT

In [105]:
df_Abs.at[166,'AbL'] = 0

In [106]:
#df_Abs['Abstract'][167]:adolescents

In [107]:
df_Abs.at[167,'AbL'] = 0

In [108]:
#df_Abs['Abstract'][169]: systematic review

In [109]:
df_Abs.at[169,'AbL'] = 0

In [110]:
#df_Abs['Abstract'][170]: SSRI plus acupuncture

In [111]:
df_Abs.at[170,'AbL'] = 0

In [112]:
#df_Abs['Abstract'][171]: adolescent

In [113]:
df_Abs.at[171,'AbL'] = 0

In [114]:
#df_Abs['Abstract'][175]: summary

In [115]:
df_Abs.at[175,'AbL'] = 0

In [116]:
#df_Abs['Abstract'][176]: protocol

In [117]:
df_Abs.at[176,'AbL'] = 0

In [118]:
#df_Abs['Abstract'][178]:colicky infants

In [119]:
df_Abs.at[178,'AbL'] = 0

In [120]:
#df_Abs['Abstract'][181]:secondary analysis of adherence and satisfaction of RCT

In [121]:
df_Abs.at[181,'AbL'] = 0

In [122]:
#df_Abs['Abstract'][182]: review

In [123]:
df_Abs.at[182,'AbL'] = 0

In [124]:
#df_Abs['Abstract'][184]: infants

In [125]:
df_Abs.at[184,'AbL'] = 0

In [126]:
#df_Abs['Abstract'][186]: not CAMs

In [127]:
df_Abs.at[186,'AbL'] = 0

In [128]:
#df_Abs['Abstract'][187]:review

In [129]:
df_Abs.at[187,'AbL'] = 0

In [82]:
#df_Abs['Abstract'][188]: CBT vs. NST

"Generalized anxiety disorder (GAD) is common in older adults; however, access to treatment may be limited, particularly in rural areas. To examine the effects of telephone-delivered cognitive behavioral therapy (CBT) compared with telephone-delivered nondirective supportive therapy (NST) in rural older adults with GAD. Randomized clinical trial in the participants' homes of 141 adults aged 60 years and older with a principal or coprincipal diagnosis of GAD who were recruited between January 27, 2011, and October 22, 2013. Telephone-delivered CBT consisted of as many as 11 sessions (9 were required) focused on recognition of anxiety symptoms, relaxation, cognitive restructuring, the use of coping statements, problem solving, worry control, behavioral activation, exposure therapy, and relapse prevention, with optional chapters on sleep and pain. Telephone-delivered NST consisted of 10 sessions focused on providing a supportive atmosphere in which participants could share and discuss the

In [131]:
df_Abs.at[188,'AbL'] = 0

In [132]:
#df_Abs['Abstract'][189]: systematic review

'To systematically review placebo-controlled randomized trials of homeopathy for psychiatric conditions. Eligible studies were identified using the following databases from database inception to April 2010: PubMed, CINAHL, PsycINFO, Hom-Inform, Cochrane CENTRAL, National Center for Complementary and Alternative Medicine grantee publications database, and ClinicalTrials.gov. Gray literature was also searched using Google, Google Scholar, the European Committee for Homeopathy, inquiries with homeopathic experts and manufacturers, and the bibliographic lists of included published studies and reviews. Search terms were as follows: (homeopath* or homoeopath*) and (placebo or sham) and (anxiety or panic or phobia or post-traumatic stress or PTSD or obsessive-compulsive disorder or fear or depress* or dysthym* or attention deficit hyperactivity or premenstrual syndrome or premenstrual disorder or premenstrual dysphoric disorder or traumatic brain injury or fibromyalgia or chronic fatigue synd

In [133]:
df_Abs.at[189,'AbL'] = 0

In [134]:
#df_Abs['Abstract'][190]: infants and children

"The objective of this clinical trial was to determine if an intervention of listening to nursery songs with integrated heartbeat sounds resulted in changes in anxiety levels in infants and children younger than 4 years in an emergent setting. This is a randomized single-blinded controlled trial. The study included 62 children younger than 4 years who were deemed by the emergency physician to need a computed tomographic (CT) scan. The experimental group (Exp) (n=30) listened to recorded children's songs with integrated heartbeat sounds (Baby Go To Sleep compact disc) before and during the head CT. The controls (Con) (n=32) had no music. Two scales to measure agitation were completed by a trained research associate: (1) a 10-cm visual analog scale (VAS) and (2) the Modified Ramsay Sedation Scale (MRSS). The research associate wore headphones blocking out all music. The research associate recorded agitation levels preintervention and postintervention on the 2 scales, whether there was su

In [135]:
df_Abs.at[190,'AbL'] = 0

In [136]:
#df_Abs['Abstract'][191]: not RCT

'Few studies have examined relationships between circadian rhythms and unipolar major depressive disorder. Further, no study to date has examined circadian markers as predictors of response to depression treatment. In the present study, we examined associations between circadian timing and its alignment with sleep and depression severity in 30 adults with major depressive disorder who completed a randomized controlled trial of two weeks of time in bed (TIB) restriction administered adjunctive to fluoxetine, with a focus on sex differences. Thirty adults with major depressive disorder received 8 weeks of fluoxetine 20-40 mgs and were randomized to 8h TIB or 6h TIB for the first 2 weeks. Participants in the 6h TIB condition were further randomized to a delayed bedtime or advanced risetime group. Circadian measures included dim light melatonin onset (DLMO) and the difference between DLMO and midsleep point (i.e., phase angle difference). Depression was assessed using the Hamilton Rating S

In [137]:
df_Abs.at[191,'AbL'] = 0

In [138]:
#df_Abs['Abstract'][192]: systematic review

" Co-morbid symptoms (for example, chronic pain, depression, anxiety, and fatigue) are particularly common in military fighters returning from the current conflicts, who have experienced physical and/or psychological trauma. These overlapping conditions cut across the boundaries of mind, brain and body, resulting in a common symptomatic and functional spectrum of physical, cognitive, psychological and behavioral effects referred to as the 'Trauma Spectrum Response' (TSR). While acupuncture has been shown to treat some of these components effectively, the current literature is often difficult to interpret, inconsistent or of variable quality. Thus, to gauge comprehensively the effectiveness of acupuncture across TSR components, a systematic review of reviews was conducted using the Samueli Institute's Rapid Evidence Assessment of the Literature (REAL©) methodology. PubMed/MEDLINE, the Cochrane Database of Systematic Reviews, EMBASE, CINAHL, and PsycInfo were searched from inception to S

In [139]:
df_Abs.at[192,'AbL'] = 0

In [140]:
#df_Abs['Abstract'][193]:combinations of St. John's Wort and tricyclic antidepressants 

"Major depressive disorder (MDD) is the most common disorder in our community. Hypericum perforatumm, is a herb with a long application history in treating depression. A controlled study to evaluate the effects of this herbal medicine in Iran did not exist. This study aimed to assess the effect of Hypericum perforatom (perforan), in combination with tricyclic antidepressants in MDD treatment. The present study was a placebo-controlled double blind randomized clinical trial including 40 patients with major depressive disorder who referred to Psychiatric Clinic and Golestan Hospital in 2011. Patients were randomly distributed in 4 blocks with 10 patients in each. 5 out of 10 patients took medicine (group I) and the others received placebo (group II). After six weeks, both groups indicated some improvement, there was also a significant degree of recovery in the perforan group (P = 0.04). The results of the current study suggested that combinations of St. John's Wort and tricyclic antidepr

In [141]:
df_Abs.at[193,'AbL'] = 0

In [144]:
#df_Abs['Abstract'][196]: review

'Objectives. The fibromyalgia syndrome (FMS) is a chronic condition with only few evidence-based complementary and alternative therapies available. This paper presents a systematic review and meta-analysis of the effectiveness of Qigong for fibromyalgia syndrome. Methods. The PubMed/MEDLINE, Cochrane Library, Embase, PsycINFO, and Cambase databases were screened in December 2012 to identify randomized controlled trials comparing Qigong to control interventions. Major outcome measures were pain and quality of life; and secondary outcomes included sleep quality, fatigue, depression, and safety. Standardized mean differences (SMD) and 95% confidence intervals were calculated. Results. Seven trials were located with a total of 395\u2009FMS patients. Analyses revealed low quality evidence for short-term improvement of pain, quality of life, and sleep quality and very low quality evidence for improvement of fatigue after Qigong for FMS, when compared to usual care. No evidence was found for 

In [145]:
df_Abs.at[196,'AbL'] = 0

In [146]:
#df_Abs['Abstract'][197]: light therapy

"Environmental (little outdoor light; low indoor lighting) and age-related physiological factors (reduced light transmission through the ocular lens, reduced mobility) contribute to a light-deprived environment for older people living in care homes. This study investigates the effect of increasing indoor light levels with blue-enriched white lighting on objective (rest-activity rhythms, performance) and self-reported (mood, sleep, alertness) measures in older people. Eighty residents (69 female), aged 86 ± 8 yrs (mean ± SD), participated (MMSE 19 ± 6). Overhead fluorescent lighting was installed in communal rooms (n=20) of seven care homes. Four weeks of blue-enriched white lighting (17000 K ≅ 900 lux) were compared with four weeks of control white lighting (4000 K ≅ 200 lux), separated by three weeks wash-out. Participants completed validated mood and sleep questionnaires, psychomotor vigilance task (PVT) and wore activity and light monitors (AWL). Rest-activity rhythms were assessed 

In [149]:
df_Abs.at[197,'AbL'] = 0

In [148]:
#df_Abs['Abstract'][198]: protocol

'The music-with-movement intervention is particularly suitable for people with dementia because their gross motor ability is preserved until the later stage of dementia. This study examines the effect of music-with-movement on reducing anxiety, sleep disturbances, and improving the wellbeing of people with dementia. This paper reports the first stage of the study - developing the intervention protocol that staff can use to teach family caregivers. A registered music therapist developed a music-with-movement protocol and taught staff of two social service centers over five weekly 1.5 h sessions, with center-in-charges (social workers and occupational therapists) and our research team joining these sessions to provide comments from their professional perspective. Each discipline had different expectations about the content; therefore, numerous meetings and discussions were held to bridge these differences and fine-tune the protocol. Few healthcare professionals doubt the merits of interd

In [150]:
df_Abs.at[198,'AbL'] = 0

In [151]:
#df_Abs['Abstract'][199]: review

'The objective of this review was to summarize evidence for the effectiveness of music therapy (MT) and to assess the quality of systematic reviews (SRs) based on randomized controlled trials (RCTs). An SR of SRs based on RCTs. Studies were eligible if they were RCTs. Studies included were those with at least one treatment group in which MT was applied. We searched the following databases from 1995 to October 1, 2012: MEDLINE via PubMed, CINAHL (Cumulative Index of Nursing and Allied Health Literature), Web of Science, Global Health Library, and Ichushi-Web. We also searched all Cochrane Database and Campbell Systematic Reviews up to October 1, 2012. Based on the International Classification of Diseases, 10th revision, we identified a disease targeted for each article. Twenty-one studies met all inclusion criteria. This study included 16 Cochrane reviews. As a whole, the quality of the articles was very good. Eight studies were about "Mental and behavioural disorders (F00-99)"; there w

In [152]:
df_Abs.at[199,'AbL'] = 0

In [154]:
#df_Abs['Abstract'][201]: review

'Acupuncture has become a popular complementary treatment in oncology, particularly as patients seek non-pharmacological alternatives to provide symptom control. A considerable body of evidence suggests that acupuncture modulates neurological processes to bring about its effects. This basic research is supported by an increasing number of positive clinical studies of varying quality. Lower quality studies have hampered the widespread acceptability of acupuncture, with some deeming the inter-personal skills of the practitioner to be more powerful than the needle or its equivalent. More recent randomised control trials (RCTs) have sought to settle this controversy, with mixed results. The literature was searched to identify, where possible, RCTs involving acupuncture and various common cancer symptoms. A potential role for acupuncture was found in the following cancer symptoms: pain, nausea and vomiting, xerostomia, hot flushes, fatigue, anxiety, depression and insomnia. Acupuncture is s

In [157]:
df_Abs.at[201,'AbL'] = 0

In [155]:
#df_Abs['Abstract'][202]: review

'Acupuncture has been suggested as a treatment for posttraumatic stress disorder (PTSD), yet its clinical effects are unclear. This review aims to estimate effects of acupuncture on PTSD symptoms, depressive symptoms, anxiety symptoms, and sleep quality for adults with PTSD. We searched 10 databases in January 2016 to identify eligible randomized controlled trials (RCTs). We performed random effects meta-analyses and examined quality of the body of evidence (QoE) using the GRADE approach to rate confidence in meta-analytic effect estimates. Seven RCTs with 709 participants met inclusion criteria. We identified very low QoE indicating significant differences favoring acupuncture (versus any comparator) at post-intervention on PTSD symptoms (standardized mean difference [SMD]\xa0=\xa0-0.80, 95% confidence interval [CI] [-1.59, -0.01], 6 RCTs), and low QoE at longer follow-up on PTSD (SMD\xa0=\xa0-0.46, 95% CI [-0.85, -0.06], 4 RCTs) and depressive symptoms (SMD\xa0=\xa0-0.56; 95% CI [-0.

In [ ]:
df_Abs.at[202,'AbL'] = 0

In [158]:
#df_Abs['Abstract'][203]: not CAM

"This study's purpose was twofold: (1) to establish adherence rates to a behavioral therapy (BT) sleep intervention and (2) to identify psychological and physical symptom predictors of adherence to the intervention in women undergoing breast cancer chemotherapy. A randomized controlled trial began 48 h before the first of four chemotherapy treatments. Women with stages I-IIIA breast cancer (n = 113) received a BT sleep intervention composed of stimulus control, modified sleep restriction (MSR), relaxation therapy (RT), and sleep hygiene counseling components. A BT plan was developed by a research nurse and each participant, reinforced on day 8, and repeated for chemotherapy cycles 2, 3, and 4. Adherence to the BT plan was measured daily; total adherence score was computed at each chemotherapy cycle by combining adherence estimates of all BT plan components. Psychological and physical symptoms over the past 7 days were measured 2 days prior to and 7 days after each chemotherapy treatmen

In [159]:
df_Abs.at[203,'AbL'] = 0

In [160]:
#df_Abs['Abstract'][204]: systematic review

'Depressive disorders are among the more common mental illnesses around the world, about 1- 3% of prepubertal children and 6% of postpubertal children and adolescents are affected. They markedly impair psychosocial development and are associated with higher rate of morbidity and mortality throughout life. Many physicians are unsure about which treatment approaches are effective and how the treatment should be planned. A systematic literature search was carried out in electronic databases and study registries and as a manual search. More than 450 studies (mostly randomized controlled trials = RCTs) were identified and summarized in evidence tables. The ensuing recommendations were agreed upon in a consensus conference. The review summarizes the evidence of complementary treatment methods. The evidence for complementary treatment methods (art and music therapy, sleep deprivation, exercise, electroconvulsive therapy, massage, transcranial magnetic stimulation, relaxation, bibliotherapy, c

In [161]:
df_Abs.at[204,'AbL'] = 0

In [162]:
#df_Abs['Abstract'][205]: interview

"People living with quadriplegia are at risk for social isolation and depression. Research with other marginalized groups has indicated that music therapy can have a positive effect on mood and social interaction. To gather descriptions of participants' experience of 2 types of group music therapy - therapeutic singing or music appreciation and relaxation - and to determine commonalities and differences between participants' experience of these 2 methods. We interviewed 20 people with quadriplegia about their experience of participating in 12 weeks of therapeutic singing (n = 10) or music appreciation and relaxation (n = 10). These methods of group music therapy were the interventions tested in a previously reported randomized controlled trial. The interview data were subjected to an inductive thematic analysis. Six main themes were generated from the interview data. Four of these were shared themes and indicated that both types of group music therapy had a positive effect on mood/ment

In [163]:
df_Abs.at[205,'AbL'] = 0

In [165]:
#df_Abs['Abstract'][207]: meta-analysis

"This study presents a meta-analysis of the effectiveness of psychological treatments for chronic nightmares using imaginal confrontation with nightmare contents (ICNC) or imagery rescripting and rehearsal (IRR). Pre-post effect sizes (Hedges' g) were calculated for the outcome measures of nightmare frequency, nights per week with nightmares, sleep quality, depression, anxiety, and PTSD severity. Fixed-effects and random-effects models were applied. High effect sizes were found for nightmare frequency (g=1.04), nights per week with nightmares (g=0.99), and PTSD severity (g=0.92). Most of the effect sizes for the secondary outcomes were moderate. One objective was to clarify whether ICNC or IRR is more important for nightmare reduction. The results indicate that a higher duration of time for ICNC is associated with greater improvements: The minutes of applied ICNC moderate the effect sizes for nightmare frequency at follow-up 2 and for nights per week with nightmares at post and follow-

In [166]:
df_Abs.at[207,'AbL'] = 0

In [167]:
#df_Abs['Abstract'][208]: secondary of RCT

"This study assessed whether elevated severities of maternal depression and disturbed maternal sleep would be associated with maternal perceptions of higher Negative Affectivity of her infant. Sixty-nine mothers participated in this study. The study was part of a larger randomized controlled study testing the efficacy of acupuncture as a treatment for depression during pregnancy. The present study focused on data collected at 6 months postpartum in a naturalistic follow-up design, using the Hamilton Rating Scale for Depression (HRSD), maternal sleep diaries (completed daily for 1 week), and the Infant Behavior Questionnaire-Revised (IBQ-R). Regression analyses revealed that (a) maternal depression severity was a significant predictor of the IBQ-R Distress and Falling Reactivity scales and (b) poor maternal sleep was a significant predictor of the IBQ-R Sadness scale. Our findings support previous findings of significant links between maternal emotional distress and perceived Negative A

In [169]:
df_Abs.at[208,'AbL'] = 0

In [170]:
#df_Abs['Abstract'][209]: adolsecent; sleep manipulation

"The relationship between inadequate sleep and mood has been well-established in adults and is supported primarily by correlational data in younger populations. Given that adolescents often experience shortened sleep on school nights, we sought to better understand the effect of experimentally induced chronic sleep restriction on adolescents' mood and mood regulation. Fifty healthy adolescents, ages 14-17, completed a 3-week sleep manipulation protocol involving a baseline week, followed by a sleep restriction (SR) condition (6.5 hr in bed per night for five nights) and healthy sleep duration (HS) condition (10 hr in bed per night for five nights). The study used a randomized, counterbalanced, crossover experimental design. Participants' sleep was monitored at home via self-report and actigraphy. At the end of each condition, participants and their parents completed questionnaires of mood and mood regulation. To assess for expectancy effects, we also analyzed parent and teen ratings of

In [171]:
df_Abs.at[209,'AbL'] = 0

In [175]:
#df_Abs['Abstract'][213]: secondary analysis

'This study presents the results of a secondary analysis of data collected during a trial of reflexology that aimed to improve health-related quality of life (HRQOL) among women with advanced breast cancer in treatment. A comparison of HRQOL (functioning, symptoms, spirituality) of those with (n = 298) and without (n = 87) distant metastasis is presented. Following the intake interview, 385 women were randomised to reflexology, lay foot manipulation or conventional care control, and were interviewed again at weeks 5 and 11. Those with distant metastasis were older, had fewer comorbid conditions, and a smaller proportion were employed. Longitudinal analysis of HRQOL at intake, 5 and 11 weeks revealed that those with distant metastasis had lower functioning and more pain; however, no differences were found on fatigue, nausea, shortness of breath, sleep quality, anxiety, depressive symptoms or spirituality. Despite advanced disease, 56% of all women in this study were below the clinical s

In [180]:
df_Abs.at[213,'AbL'] = 0

0

In [177]:
#df_Abs['Abstract'][214]: summary

'Although treatment research for NES remains limited, several options are available for patients whose symptoms require clinical attention. Pharmacotherapy has received the most empirical support of the proposed treatments. Controlled trials are needed to confirm the initial results from pilot studies with CBT, behavioral therapy, and phototherapy, and an extended controlled trial of progressive muscle relaxation would be useful. In their comprehensive review of the field, Striegel-Moore and colleagues have questioned the clinical utility of NES as a diagnostic entity and stress the very limited nature of treatment studies to date. Research in this field has to provide a systematic examination of the approaches described here, as well as others yet to be identified. This pursuit seems warranted given that persons suffering with the cluster of symptoms identified as NES are approaching health care providers for relief and are often frustrated by the lack of recognition of this syndrome.

In [179]:
df_Abs.at[214,'AbL'] = 0

In [181]:
#df_Abs['Abstract'][215]: secondary of RCT

'To describe self-reported menopausal symptom priorities and their association with demographics and other symptoms among participants in an intervention trial for vasomotor symptoms (VMS). Cross-sectional study embedded in the MsFLASH 02 trial, a three-by-two factorial design of yoga vs. exercise vs. usual activity and omega-3-fatty acid vs. placebo. At baseline, women (n\u2009=\u2009354) completed hot flush diaries, a card sort task to prioritize symptoms they would most like to alleviate, and standardized questionnaires. The most common symptom priorities were: VMS (n\u2009=\u2009322), sleep (n\u2009=\u2009191), concentration (n\u2009=\u2009140), and fatigue (n\u2009=\u2009116). In multivariate models, women who chose VMS as their top priority symptom (n\u2009=\u2009210) reported significantly greater VMS severity (p\u2009=\u20090.004) and never smoking (p\u2009=\u20090.012), and women who chose sleep as their top priority symptom (n\u2009=\u2009100) were more educated (p\u2009≤\u20

In [182]:
df_Abs.at[215,'AbL'] = 0

In [185]:
#df_Abs['Abstract'][219]:summary

'Seventy percentage of perimenopausal and early postmenopausal women will experience menopause symptoms. Primary menopause symptoms in Western countries included hot flashes, insomnia, somatic pain, depression, and fatigue. Hot flashes were most commonly treated. Menopausal hormone replacement therapy (HRT) continues to have a clinical role in the management of vasomotor symptoms, but since 2002 there has been a marked global decline in its use due to concerns about the risks and benefits of HRT; consequently many women with menopause symptoms are now seeking alternatives including acupuncture. Acupuncture has a long tradition of use for the treatment of different menopause symptoms. Its effectiveness has been studied for natural menopause or chemical and surgery induced menopause. Here we provide an update on recent advances in the field for clinicians. The recent systematic reviews on acupuncture in menopausal symptoms suggest that acupuncture is an effective and valuable option for 

In [191]:
df_Abs.at[219,'AbL'] = 0

0

In [189]:
#df_Abs['Abstract'][222]: review

"Chronic pain is defined as pain lasting beyond normal tissue healing time, generally taken to be 12 weeks. It contributes to disability, anxiety, depression, sleep disturbances, poor quality of life, and healthcare costs. Chronic pain has a weighted mean prevalence in adults of 20%.For many years, the treatment choice for chronic pain included recommendations for rest and inactivity. However, exercise may have specific benefits in reducing the severity of chronic pain, as well as more general benefits associated with improved overall physical and mental health, and physical functioning.Physical activity and exercise programmes are increasingly being promoted and offered in various healthcare systems, and for a variety of chronic pain conditions. It is therefore important at this stage to establish the efficacy and safety of these programmes, and furthermore to address the critical factors that determine their success or failure. To provide an overview of Cochrane Reviews of adults wit

In [192]:
df_Abs.at[222,'AbL'] = 0

In [194]:
#df_Abs['Abstract'][223]:duplicate with [222]

"Chronic pain is defined as pain lasting beyond normal tissue healing time, generally taken to be 12 weeks. It contributes to disability, anxiety, depression, sleep disturbances, poor quality of life, and healthcare costs. Chronic pain has a weighted mean prevalence in adults of 20%.For many years, the treatment choice for chronic pain included recommendations for rest and inactivity. However, exercise may have specific benefits in reducing the severity of chronic pain, as well as more general benefits associated with improved overall physical and mental health, and physical functioning.Physical activity and exercise programmes are increasingly being promoted and offered in various healthcare systems, and for a variety of chronic pain conditions. It is therefore important at this stage to establish the efficacy and safety of these programmes, and furthermore to address the critical factors that determine their success or failure. To provide an overview of Cochrane Reviews of adults wit

In [196]:
df_Abs.at[223,'AbL'] = 0

In [199]:
#df_Abs['Abstract'][226]: review

'Sleep deprivation is associated with an elevated risk of various diseases and leads to a poor quality of life and negative socioeconomic consequences. Sleep inducers such as drugs and herbal medicines may often lead to dependence and other side effects. L-Theanine (γ-glutamylethylamide), an amino acid naturally found abundant in tea leaves, has anxiolytic effects via the induction of α brain waves without additive and other side effects associated with conventional sleep inducers. Anxiolysis is required for the initiation of high-quality sleep. In this study, we review the mechanism(s), safety, and efficacy of L-theanine. Collectively, sleep studies based on an actigraph, the obstructive sleep apnea (OSA) sleep inventory questionnaire, wakeup after sleep onset (WASO) and automatic nervous system (ANS) assessment, sympathetic and parasympathetic nerve activities, and a pediatric sleep questionnaire (PSQ) suggest that the administration of 200 mg of L-theanine before bed may support imp

In [200]:
df_Abs.at[226,'AbL'] = 0

In [201]:
#df_Abs['Abstract'][227]: review

"Chronic fatigue syndrome (CFS) is characterised by persistent, medically unexplained fatigue, as well as symptoms such as musculoskeletal pain, sleep disturbance, headaches and impaired concentration and short-term memory. CFS presents as a common, debilitating and serious health problem. Treatment may include physical interventions, such as exercise therapy, which was last reviewed in 2004. The objective of this review was to determine the effects of exercise therapy (ET) for patients with CFS as compared with any other intervention or control.• Exercise therapy versus 'passive control' (e.g. treatment as usual, waiting-list control, relaxation, flexibility).• Exercise therapy versus other active treatment (e.g. cognitive-behavioural therapy (CBT), cognitive treatment, supportive therapy, pacing, pharmacological therapy such as antidepressants).• Exercise therapy in combination with other specified treatment strategies versus other specified treatment strategies (e.g. exercise combin

In [202]:
df_Abs.at[227,'AbL'] = 0

In [204]:
#df_Abs['Abstract'][228]: duplicate with [227]

"Chronic fatigue syndrome (CFS) is characterised by persistent, medically unexplained fatigue, as well as symptoms such as musculoskeletal pain, sleep disturbance, headaches and impaired concentration and short-term memory. CFS presents as a common, debilitating and serious health problem. Treatment may include physical interventions, such as exercise therapy, which was last reviewed in 2004. The objective of this review was to determine the effects of exercise therapy (ET) for patients with CFS as compared with any other intervention or control.• Exercise therapy versus 'passive control' (e.g. treatment as usual, waiting-list control, relaxation, flexibility).• Exercise therapy versus other active treatment (e.g. cognitive-behavioural therapy (CBT), cognitive treatment, supportive therapy, pacing, pharmacological therapy such as antidepressants).• Exercise therapy in combination with other specified treatment strategies versus other specified treatment strategies (e.g. exercise combin

In [205]:
df_Abs.at[228,'AbL'] = 0

In [206]:
#df_Abs['Abstract'][229]: not CAM RCT

'Women with breast cancer frequently report distressing symptoms during and after treatment that can significantly erode quality of life (QOL). Symptom burden among women with breast cancer is of complex etiology and is likely influenced by disease, treatment, and environmental factors as well as individual genetic differences. The purpose of the present study was to examine the relationships between genetic polymorphisms within Neurotrophic tyrosine kinase receptor 1 (NTRK1), Neurotrophic tyrosine kinase receptor 2 (NTRK2), and catechol-O-methyltransferase ( COMT) and patient symptom burden of QOL, pain, fatigue, anxiety, depression, and sleep disturbance before, during, and after treatment for breast cancer in a subset of participants ( N = 51) in a randomized clinical trial of a novel symptom-management modality for women with breast cancer undergoing chemotherapy. Patients were recruited at the time of initial breast cancer diagnosis and completed all survey measures at the time of

In [208]:
df_Abs.at[229,'AbL'] = 0

In [209]:
#df_Abs['Abstract'][230]: CBT-T vs. NST-T

'To examine the long-term effects of telephone-delivered cognitive-behavioral therapy (CBT-T) compared with nondirective supportive therapy (NST-T) in rural older adults with generalized anxiety disorder (GAD). 141 adults aged 60 years and older with a principal/co-principal diagnosis of GAD were randomized to either CBT-T or NST-T. CBT-T consisted of up to 11 sessions (9 were required) focused on recognition of anxiety symptoms, relaxation, cognitive restructuring and use of coping statements, problem-solving, worry control, behavioral activation, exposure therapy, and relapse prevention, with optional chapters on sleep and pain. NST-T consisted of 10 sessions focused on providing a supportive atmosphere in which participants could share and discuss their feelings and did not provide any direct suggestions. Primary outcomes included interviewer-rated anxiety severity and self-report worry severity measured at 9 months and 15 months after randomization. Mood-specific secondary outcomes

In [ ]:
df_Abs.at[230,'AbL'] = 0

In [210]:
#df_Abs['Abstract'][231]: secondary, qualitative analysis

'The aim of this study was to examine the quality of life of breast cancer patients medicated with estrogen antagonists, 2 years after having acupuncture treatment for hot flashes. Our sample was taken from women who had recently participated in a randomized controlled trial investigating the effects of acupuncture on hot flashes, a side effect of estrogen-antagonist treatment. Forty-one women from the true acupuncture treatment group and 41 women from the control group (sham acupuncture), who had 2 years previously received a course of 15 acupuncture treatments over a period of 10 weeks, were asked to answer an open question. The question, "Would you like to share your thoughts and experiences related to your breast cancer diagnosis, treatments or anything else?" was by being open, broad, and nonspecific, intended to stimulate subjective information, which was not included in the original, or future quantitative studies. Qualitative data were analyzed using systematic text condensatio

In [212]:
df_Abs.at[231,'AbL'] = 0

In [213]:
#df_Abs['Abstract'][232]:duplicate with [227]

"Chronic fatigue syndrome (CFS) is characterised by persistent, medically unexplained fatigue, as well as symptoms such as musculoskeletal pain, sleep disturbance, headaches and impaired concentration and short-term memory. CFS presents as a common, debilitating and serious health problem. Treatment may include physical interventions, such as exercise therapy, which was last reviewed in 2004. The objective of this review was to determine the effects of exercise therapy (ET) for patients with CFS as compared with any other intervention or control.• Exercise therapy versus 'passive control' (e.g. treatment as usual, waiting-list control, relaxation, flexibility).• Exercise therapy versus other active treatment (e.g. cognitive-behavioural therapy (CBT), cognitive treatment, supportive therapy, pacing, pharmacological therapy such as antidepressants).• Exercise therapy in combination with other specified treatment strategies versus other specified treatment strategies (e.g. exercise combin

In [215]:
df_Abs.at[232,'AbL'] = 0

In [217]:
#df_Abs['Abstract'][234]:duplicate with [227]

"Chronic fatigue syndrome (CFS) is characterised by persistent, medically unexplained fatigue, as well as symptoms such as musculoskeletal pain, sleep disturbance, headaches and impaired concentration and short-term memory. CFS presents as a common, debilitating and serious health problem. Treatment may include physical interventions, such as exercise therapy, which was last reviewed in 2004. The objective of this review was to determine the effects of exercise therapy (ET) for patients with CFS as compared with any other intervention or control.• Exercise therapy versus 'passive control' (e.g. treatment as usual, waiting-list control, relaxation, flexibility).• Exercise therapy versus other active treatment (e.g. cognitive-behavioural therapy (CBT), cognitive treatment, supportive therapy, pacing, pharmacological therapy such as antidepressants).• Exercise therapy in combination with other specified treatment strategies versus other specified treatment strategies (e.g. exercise combin

In [219]:
df_Abs.at[234,'AbL'] = 0

In [221]:
#df_Abs['Abstract'][236] : protocol

'The purpose of this study is to conduct a basic analysis of the effectiveness and safety of electroacupuncture in the treatment of painful diabetic neuropathy (PDN) as compared to placebo and usual care and to evaluate the feasibility of large-scale clinical research. This study is a protocol for a three-armed, randomized, patient-assessor-blinded (to the type of treatment), controlled pilot trial. Forty-five participants with a ≥ six month history of PDN and a mean weekly pain score of ≥ 4 on the 11-point Pain Intensity Numerical Rating Scale (PI-NRS) will be assigned to the electroacupuncture group (n = 15), sham group (n = 15) or usual care group (n = 15). The participants assigned to the electroacupuncture group will receive electroacupuncture (remaining for 30 minutes with a mixed current of 2 Hz/120 Hz and 80% of the bearable intensity) at 12 standard acupuncture points (bilateral ST36, GB39, SP9, SP6, LR3 and GB41) twice per week for eight weeks (a total of 16 sessions) as well

In [222]:
df_Abs.at[236,'AbL'] = 0

In [223]:
#df_Abs['Abstract'][237]: review

'The present paper reviews recent studies on the clinical presentation of generalized anxiety disorder (GAD) and the current level of evidence for treatment with medications and psychotherapy. An apprehensive state of mind, regardless of the quality of stimuli, is at the core of GAD, causing a pervasive cognitive dysfunction that is separate from that seen in depression and obsessive compulsive disorder. When treatments reduce anxiety-related symptoms and insomnia, patients report a restoration of social functioning, probably as a consequence of improved decision-making, cooperative skills, and risk assessment. Late-onset GAD symptoms in the elderly may be caused or aggravated by cerebrovascular events, loneliness, bereavement, substance use, and the prospect of death. Insomnia in GAD may be primary or caused by common medications. Practice guidelines fail to capture the needs of the entire range of patients with GAD as they build on patients without comorbidity who have been selected 

In [225]:
df_Abs.at[237,'AbL'] = 0

In [226]:
#df_Abs['Abstract'][238]: systematic review

'This systematic review of randomized controlled trials aimed to examine the effectiveness of dry needling in the treatment of myofascial trigger points and to explore the impact of specific aspects of the technique on its effectiveness. Relevant studies published between 2000 and 2015 were identified by searching PubMed, Scopus, The Cochrane Library and Physiotherapy Evidence Database. Studies identified by electronic searches were screened against a set of pre-defined inclusion criteria. Fifteen studies were included in this systematic review. The main outcomes that were measured were pain, range of motion, disability, depression and quality of life. The results suggest that dry needling is effective in the short term for pain relief, increase range of motion and improve quality of life when compared to no intervention/sham/placebo. There is insufficient evidence on its effect on disability, analgesic medication intake and sleep quality. Despite some evidence for a positive effect in

In [227]:
df_Abs.at[238,'AbL'] = 0

In [228]:
#df_Abs['Abstract'][239]: review

'Potential benefits and risks of early (≤30 days from stroke onset) selective serotonin reuptake inhibitors (SSRIs) treatment for neurologic functional recovery after stroke are not fully understood. We searched PubMed, Embase, and the Cochrane Library to identify randomized controlled trials that assessed SSRI medications during the initial ictus after stroke versus placebo. Primary outcome was decrease in National Institutes of Health Stroke Scale (NIHSS) score. Secondary outcomes included the improvement of Barthel index, functional independence (modified Rankin Scale score 0-2 at the end of follow-up), the incidence of depression, and adverse events including diarrhea, insomnia, hepatic enzyme disorders, seizure, and intracranial hemorrhage. We used fixed effects models or random effects models to estimate weighted mean differences (WMDs) and relative risks (RRs) with 95% confidence intervals (CIs) according to heterogeneity. Eight trials were included, with 1549 patients. Compared

In [229]:
df_Abs.at[239,'AbL'] = 0

In [230]:
#df_Abs['Abstract'][240]: CBT

'Chronic pain is a common condition among older adults. While cognitive behaviour therapy (CBT) has been tested in numerous studies on adults and children there are fewer studies on older persons. The objective of this study was to investigate the effects of a six-session CBT group treatment for older persons with chronic pain. As a secondary aim we investigated whether treatment credibility was associated with outcome. We included 21 persons (mean age = 72.0 years) who were randomly allocated to either a waitlist condition or treatment consisting of applied relaxation, with the addition of problem solving, assertiveness, communication strategies, sleep management, and relapse prevention. Few statistically significant effects were found on measures of pain, mood, anxiety, and quality of life; however, a significant treatment effect with a between group effect size of d = 1.0 was observed with respect to perceived ability to function despite the discomfort of pain. The study provides so

In [231]:
df_Abs.at[240,'AbL'] = 0

In [232]:
#df_Abs['Abstract'][241]: not RCT

'As the number of people with diabetes is increasing rapidly worldwide, a more thorough understanding of the psychosocial aspects of living with this condition has become an important health care priority. While our knowledge has grown substantially over the past two decades with respect to the physical, emotional and social difficulties that people with diabetes may encounter, many important issues remain to be elucidated. Under the umbrella of the Diabetes MILES (Management and Impact for Long-term Empowerment and Success) Study International Collaborative, Diabetes MILES--The Netherlands aims to examine how Dutch adults with diabetes manage their condition and how it affects their lives. Topics of special interest in Diabetes MILES--The Netherlands include subtypes of depression, Type D personality, mindfulness, sleep and sexual functioning. Diabetes MILES--The Netherlands was designed as a national online observational study among adults with diabetes. In addition to a main set of 

In [234]:
df_Abs.at[241,'AbL'] = 0

In [235]:
#df_Abs['Abstract'][242]: not CAM

'Moderate exercise training during the pregnancy benefits both the pregnant woman and the fetus. The research papers consulted have linked physical activity with the reduction of the number of cesareans, instrumental delivery and with a more physiological delivery. It also prevents excessive weight gain in women and decreases the risk of gestational diabetes and high blood pressure. The aim of this research is to know if an exercise program of moderate character with Water Study Exercise Pregnant (SWEP) method, performed in an aquatic environment, contributes to have more favorable results in the perinatal period, both for women and baby. The design is a randomized clinical trial. The sample will consist of 364 pregnant women, with a total universe of 6,579 births occurred in Granada (Spain) during 2014. The sample was divided into two groups, intervention group and control group. The activity will be carried out in the water sporting facilities of the Faculty of Sports Science of the 

In [236]:
df_Abs.at[242,'AbL'] = 0

In [237]:
#df_Abs['Abstract'][243]: not CAM

"This study evaluated the efficacy of a comprehensive stress management program in reducing perceived stress among women who reported moderate-to-high stress levels. A total of 562 highly motivated females, aged 25-45, with moderate to high stress levels, were enrolled in a 14-week study. Participants were randomized into one of three groups: Group 1 included Internet-based coaching focusing on behavior modification, daily use of proprietary olfactive-based personal care products, and periodic feedback reports; Group 2 consisted of only online coaching; and Group 3 had no active stress management program. Participants in the three groups filled out validated psychometric assessments at baseline and throughout the study period. Several outcomes including Perceived Stress Scale (PSS), Profile of Mood States (POMS), St Mary's Hospital Sleep Questionnaire (SMS), the Trier Inventory of Chronic Stress (TICS), Spielberger State-Trait Anxiety Inventory (STAI), Short-Form-36 (SF-36) and the Wor

In [238]:
df_Abs.at[243,'AbL'] = 0

In [240]:
df_Abs.to_pickle('C:/Users/CAM2-1.pkl')
#df_Abs.to_pickle('C:/Users/kirak/Documents/School/USF/HS651/CAM2.pkl')

In [241]:
df_Abs = pd.read_pickle('C:/Users/CAM2-1.pkl')

In [242]:
df_Abs.head(10)

,Title,JournalInfo,Abstract,TiL,AbL
0,"""Qigong/Tai Chi Easy for fatigue in breast can...",Contemp Clin Trials,Breast cancer survivors (BCSs) often report fa...,1,0
1,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1,1
2,"""Cognitive behavioral therapy vs. Tai Chi for ...",Sleep,To investigate the comparative efficacy of cog...,1,1
3,"""Tai chi/yoga reduces prenatal depression, anx...",Complement Ther Clin Pract,Ninety-two prenatally depressed pregnant women...,1,1
4,"""Efficacy of rehabilitation with Tai Ji Quan i...",Complement Ther Clin Pract,Fibromyalgia Syndrome (FMS) is characterized b...,1,1
5,"""Randomized controlled trial of Qigong/Tai Chi...",Ann Behav Med,Many breast cancer survivors experience fatigu...,1,1
6,"""Subjective perceived impact of Tai Chi traini...",BMC Complement Altern Med,Evidence from quantitative studies suggest tha...,1,1
7,"""Design and methods of the Gentle Cardiac Reha...",Contemp Clin Trials,Cardiac rehabilitation (CR) programs reduce ov...,1,0
8,"""Long-term Exercise After Pulmonary Rehabilita...",Contemp Clin Trials,Persons with chronic obstructive pulmonary dis...,1,0
9,"""Patient Engagement in Randomized Controlled T...",Rev Recent Clin Trials,Physicians encounter various symptom-based com...,1,0


In [244]:
df_Abs.AbL.value_counts()

0    126
1    118
Name: AbL, dtype: int64

In [247]:
del df_Abs['Title']

In [248]:
del df_Abs['JournalInfo']

In [249]:
del df_Abs['TiL']

In [250]:
df_Abs.head()

,Abstract,AbL
0,Breast cancer survivors (BCSs) often report fa...,0
1,Purpose Cognitive behavioral therapy for insom...,1
2,To investigate the comparative efficacy of cog...,1
3,Ninety-two prenatally depressed pregnant women...,1
4,Fibromyalgia Syndrome (FMS) is characterized b...,1


In [251]:
df_Abs.to_pickle('C:/Users/CAM2-1-1.pkl')

In [245]:
#df_Abs1 = pd.read_pickle('C:/Users/CAM2.pkl')

In [55]:
#df_Abs1.head()

In [56]:
#df_Abs1.AbL.value_counts()

# Concatenate pkl CAM2 and CAM2-1-1

In [9]:
import pandas as pd

In [10]:
df_Abs_Cam2 = pd.read_pickle('C:/Users/CAM2-1-1.pkl')
df_Abs_Cam2

,Abstract,AbL
0,Breast cancer survivors (BCSs) often report fa...,0
1,Purpose Cognitive behavioral therapy for insom...,1
2,To investigate the comparative efficacy of cog...,1
3,Ninety-two prenatally depressed pregnant women...,1
4,Fibromyalgia Syndrome (FMS) is characterized b...,1
...,...,...
239,Potential benefits and risks of early (≤30 day...,0
240,Chronic pain is a common condition among older...,0
241,As the number of people with diabetes is incre...,0
242,Moderate exercise training during the pregnanc...,0


In [11]:
df_Abs_Cam21 = pd.read_pickle('C:/Users/CAM2.pkl')
df_Abs_Cam21

,Abstract,AbL
0,and Purpose: There were studies showing the po...,1
1,"Tai Chi, a kind of physical exercise, may act ...",1
2,To determine the effectiveness of tai chi inte...,1
3,This study aimed to investigate the effects of...,1
4,Cancer treatment can be a great source of anxi...,1
...,...,...
145,Fibromyalgia is a condition characterised by c...,1
146,Mild cognitive impairment (MCI) is cognitive d...,0
147,There is no sufficient evidence on the effecti...,0
148,Many of the currently available treatments for...,0


In [5]:
dfstocomb = [df_Abs_Cam2,df_Abs_Cam21]
combinedDFs = pd.concat(dfstocomb,axis=0)
combinedDFs.head()

,Abstract,AbL
0,Breast cancer survivors (BCSs) often report fa...,0
1,Purpose Cognitive behavioral therapy for insom...,1
2,To investigate the comparative efficacy of cog...,1
3,Ninety-two prenatally depressed pregnant women...,1
4,Fibromyalgia Syndrome (FMS) is characterized b...,1


In [6]:
combinedDFs.isna().any()

Abstract    False
AbL         False
dtype: bool

In [8]:
combinedDFs.to_pickle('C:/users/Combined_df.pkl')

# Split

In [61]:
df_valid = combinedDFs.sample(frac = 0.2, random_state = 42)

In [63]:
df_valid.to_pickle('C:/users/Combined_valid.pkl')
#df_valid.to_pickle('C:/Users/kirak/Documents/School/USF/HS651/CAM2_valid.pkl')

In [64]:
df_train = combinedDFs.drop(df_valid.index)

In [65]:
df_train.to_pickle('C:/users/Combined_train.pkl')
#df_train.to_pickle('C:/Users/kirak/Documents/School/USF/HS651/CAM2_train.pkl')

In [66]:
df_train.AbL.value_counts()

1    148
0    123
Name: AbL, dtype: int64